Описание всех таблиц:

➔ child_mortality_rural_1990_2021.xls — число умерших на первом году жизни детей за год, по всем регионам, в сельской местности.

➔ child_mortality_urban_1990_2021.xls — число умерших на первом году жизни детей за год, по всем регионам, в городской местности.

➔ disabled_total_by_age_2017_2022.csv — число людей с инвалидностью по регионам, по месяцам, по возрастным группам.

➔ morbidity_2005_2020_age_disease.xls — заболеваемость на 100 тыс. человек населения, по возрастным группам и группам заболеваний.

➔ poverty_percent_by_regions_1992_2020.csv — процент людей, живущих за чертой бедности (с денежными доходами ниже величины
прожиточного минимума), оценка за год по регионам.

➔ welfare_expense_share_2015_2020 — расходы на социальную политику от общих расходов бюджета региона, % в год*.

➔ cash_real_income_wages_2015_2020 — среднедушевые и реальные денежные доходы населения, номинальная и реальная начисленная
зарплата, по регионам*.

➔ poverty_socdem_20*.xls — распределение малоимущего населения по социально-демографическим группам (дети, трудящиеся,
пенсионеры) за 2017–2020 гг., по регионам.

➔ housing_2020 — характеристика жилищных условий домохозяйств. Оценка домохозяйствами состояния занимаемого ими жилого
помещения, обследование 2020 года*.

➔ population.xlsx — численность населения по регионам и федеральным округам на 1 января каждого года за 1999–2022 гг.

➔ gross_regional_product_1996_2020.xls — валовой региональный продукт на душу населения, в рублях.

<img src=titul_picture.jpg width=1300px height=450> 

# <center>Определение уязвимых групп населения #

Задача: 


* кластеризовать регионы России и определить, какие из них наиболее остро нуждаются в помощи малообеспеченным/неблагополучным слоям населения;
* описать группы населения, сталкивающиеся с бедностью;
* определить:
    - влияет ли число детей, пенсионеров и других социально уязвимых групп на уровень бедности в регионе;
    - связаны ли уровень бедности/социального неблагополучия с производством и потреблением в регионе;
    - какие ещё зависимости можно наблюдать относительно социально незащищённых слоёв населения.


In [1]:
# Все необходимые библиотеки для работы.

import numpy as np
import pandas as pd
import re
import seaborn as sns
from matplotlib import pyplot as plt
import plotly 
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
%matplotlib inline
plt.style.use('bmh')
from sklearn.cluster import KMeans

from sklearn.preprocessing import LabelEncoder,LabelBinarizer,OneHotEncoder,RobustScaler,StandardScaler

In [2]:
dct_city = {'Архангельская область':'Архангельская обл.(кроме НАО)',
'Архангельская область (кроме Ненецкого автономного округа)':'Архангельская обл.(кроме НАО)',             
'Город Москва столица Российской Федерации город федерального значения':'Москва',
'Город Санкт-Петербург город федерального значения':'Санкт-Петербург',
'Город федерального значения Севастополь':'Севастополь',
'Кемеровская область - Кузбасс':'Кемеровская область',
'Ненецкий автономный округ (Архангельская область)': 'Ненецкий АО.(Арх.обл.)',
'Ненецкий автономный округ':'Ненецкий АО.(Арх.обл.)',
'Республика Адыгея (Адыгея)':'Республика Адыгея',
'Республика Татарстан (Татарстан)':'Республика Татарстан',
'Тюменская область':'Тюменская обл.(кроме ХМАО,ЯНАО)',
'Ханты-Мансийский автономный округ - Югра (Тюменская область)':'ХМАО(Тюменская обл.)',
'Чувашская Республика - Чувашия':'Чувашская Республика',
'Ямало-Ненецкий автономный округ (Тюменская область)':'ЯНАО(Тюменская обл.)',
'Ямало-Ненецкий автономный округ':'ЯНАО(Тюменская обл.)',
'г. Москва':'Москва',
'г. Санкт-Петербург':'Санкт-Петербург',
'г. Севастополь':'Севастополь',
'город Москва':'Москва',
'город Санкт-Петербург':'Санкт-Петербург',
'город Севастополь':'Севастополь',
'Республика Северная Осетия - Алания':'Республика Северная Осетия-Алания',
'Республика Северная Осетия — Алания': 'Республика Северная Осетия-Алания',
'Ямало-Ненецкий  автономный округ':'ЯНАО(Тюменская обл.)',
'Ханты-Мансийский  автономный округ - Югра':'ХМАО(Тюменская обл.)',
'Ханты-Мансийский автономный округ — Югра':'ХМАО(Тюменская обл.)',
'Ханты-Мансийский автономный округ':'ХМАО(Тюменская обл.)',
'Архангельская область без автономного округа':'Архангельская обл.(кроме НАО)',
'Тюменская область без автономных округов':'Тюменская обл.(кроме ХМАО,ЯНАО)',
'Архангельская область без авт. округа':'Архангельская обл.(кроме НАО)',
'Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)':'Тюменская обл.(кроме ХМАО,ЯНАО)',
'Тюменская область без авт. округов':'Тюменская обл.(кроме ХМАО,ЯНАО)'}

In [3]:
data = pd.read_excel('./social_russia_data/poverty_socdem_2020.xls')

data = data.iloc[1:,0].reset_index(drop=True)
data = pd.DataFrame({'Регионы':data})
data['Регионы'] = data['Регионы'].apply(lambda x : x.strip())
data

,Регионы
0,Белгородская область
1,Брянская область
2,Владимирская область
3,Воронежская область
4,Ивановская область
...,...
82,Амурская область
83,Магаданская область
84,Сахалинская область
85,Еврейская автономная область


Удалим два региона, Тюменская и Архангельские области, т.к. они ниже расписаны.

In [4]:
sns.heatmap(data.corr(numeric_only=True))

ValueError: zero-size array to reduction operation fmin which has no identity

In [ ]:
data.drop([20,61],inplace=True)
data = data.reset_index(drop=True)
data

Итак у нас есть 85 регионов России. С ними мы будем работать. Для дальнейшей работы будем приводить к общим названиям города.

In [121]:
data['Регионы'] = data['Регионы'].apply(lambda x :dct_city[x] if x in dct_city else x)

# <center> Обработка датасета

## 1) Численность населения

In [122]:
# Численность населения по годам, расчитывается на 1.01 каждого года
data_population = pd.read_excel('./social_russia_data/population.xlsx')
data_population = data_population.iloc[2:,:].drop(['Unnamed: 1'],axis=1)
col = ['Регион']
col +=[x for x in range(1999,2023,1)]

data_population.columns = col
data_population['Регион'] = data_population['Регион'].shift(1) 
mask = data_population[data_population['Регион']=='Российская Федерация'].T.reset_index()
mask = mask.iloc[1:,:].rename(columns={'index':'Год',3:'Численность в млн.'})


data_fig = [go.Scatter(x=mask['Год'],
                 y=mask['Численность в млн.'],
                 name='Популяция'),
                 go.Scatter(x=mask[mask['Год']==2020]['Год'],y=mask[mask['Год']==2020]['Численность в млн.'],
                         name=mask[mask['Год']==2020]['Численность в млн.'].iloc[0],
                         marker=dict(color='Green',
                         size=10))] 

data_layout = go.Layout(title='Численность России в период с 1999 по 2021',
                        legend=dict(x=5),
                        xaxis=dict(title='Год'),
                        yaxis=dict(title='Млн.человек'))

fig = go.Figure(data=data_fig,
                layout=data_layout)

fig.show()

Численность видим, что был резкий рост в 2015г. Наверняка это связано с тем, что присоединились новые регионы.

In [123]:
# Численность населения 2020
data_population = pd.read_excel('./social_russia_data/population.xlsx')
data_population_2020 = data_population.iloc[2:,[0,23]]

data_population_2020.columns = ['region','Численность населения']

data_population_2020.iloc[:,1] = data_population_2020.iloc[:,1].shift(-1)
data_population_2020.dropna(inplace=True)
data_population_2020 = data_population_2020.reset_index(drop=True)

data_population_2020

,region,Численность населения
0,Российская Федерация,146748590
1,Центральный федеральный округ,39433556
2,Белгородская область,1549151
3,Брянская область,1192491
4,Владимирская область,1358416
...,...,...
240,Амурская область,790044
241,Магаданская область,140149
242,Сахалинская область,488257
243,Еврейская автономная область,158305


In [124]:
mask_population = data_population_2020[(data_population_2020['region']).str.contains('федеральный')]

ind = mask_population.index
cols = list(mask_population['region'])


for num,value in enumerate(range(len(ind))):
    if num == len(mask_population.index)-1:
        v1 = ind[num]
        v2 = data_population_2020.index[-1] 
        data_population_2020.loc[v1:(v2),'Федеральный округ'] = cols[num]
    else:
        v1 = ind[num]
        v2 = ind[num+1]
        data_population_2020.loc[v1:(v2-1),'Федеральный округ'] = cols[num]


    


data_population_2020

,region,Численность населения,Федеральный округ
0,Российская Федерация,146748590,NaN
1,Центральный федеральный округ,39433556,Центральный федеральный округ
2,Белгородская область,1549151,Центральный федеральный округ
3,Брянская область,1192491,Центральный федеральный округ
4,Владимирская область,1358416,Центральный федеральный округ
...,...,...,...
240,Амурская область,790044,Дальневосточный федеральный округ
241,Магаданская область,140149,Дальневосточный федеральный округ
242,Сахалинская область,488257,Дальневосточный федеральный округ
243,Еврейская автономная область,158305,Дальневосточный федеральный округ


In [125]:
data_population_2020.drop([23,68],axis=0,inplace=True)

In [126]:

data_population_2020['region'] = data_population_2020['region'].apply(lambda x :dct_city[x] if x in dct_city else x)
data_population_2020

,region,Численность населения,Федеральный округ
0,Российская Федерация,146748590,NaN
1,Центральный федеральный округ,39433556,Центральный федеральный округ
2,Белгородская область,1549151,Центральный федеральный округ
3,Брянская область,1192491,Центральный федеральный округ
4,Владимирская область,1358416,Центральный федеральный округ
...,...,...,...
240,Амурская область,790044,Дальневосточный федеральный округ
241,Магаданская область,140149,Дальневосточный федеральный округ
242,Сахалинская область,488257,Дальневосточный федеральный округ
243,Еврейская автономная область,158305,Дальневосточный федеральный округ


In [127]:
data = data.merge(right=data_population_2020,
           how='left',
           left_on='Регионы',
           right_on='region').drop(['region'],axis=1)

data.sort_values(by='Численность населения',ascending=False,ignore_index=True,inplace=True)
data

,Регионы,Численность населения,Федеральный округ
0,Москва,12678079,Центральный федеральный округ
1,Московская область,7690863,Центральный федеральный округ
2,Краснодарский край,5675462,Южный федеральный округ (с 29.07.2016)
3,Санкт-Петербург,5398064,Северо-Западный федеральный округ
4,Свердловская область,4310681,Уральский федеральный округ
...,...,...,...
80,Республика Алтай,220181,Сибирский федеральный округ
81,Еврейская автономная область,158305,Дальневосточный федеральный округ
82,Магаданская область,140149,Дальневосточный федеральный округ
83,Чукотский автономный округ,50288,Дальневосточный федеральный округ


Итак, у нас теперь имеется численность населения и федеральный округ. Для более детальной реализации графиков, Над регионами миллиониками будет отображаться '★' и расскрасим в разные цвета округа, что бы было более наглядно видно на графиках есть ли какая-то зависимость от региона или от численности населения.

In [128]:
colors = [ '#1F77B4', # Azure Blue
    '#FF7F0E', # Tangerine
    '#2CA02C', # Forest Green
    '#D62728', # Ruby Red
    '#9467BD', # Violet Iris
    '#8C564B', # Sandstone
    '#E377C2', # Coral Blush
    '#BCBD22'  # Lime Yellow
]
dct_colors = dict()
for num,value in enumerate(data['Федеральный округ'].unique()):
    dct_colors[value]=colors[num]

label_region = data['Численность населения'].apply(lambda x: '★' if x>999999 else '') 

data.insert(1,'метка региона',label_region)
data

,Регионы,метка региона,Численность населения,Федеральный округ
0,Москва,★,12678079,Центральный федеральный округ
1,Московская область,★,7690863,Центральный федеральный округ
2,Краснодарский край,★,5675462,Южный федеральный округ (с 29.07.2016)
3,Санкт-Петербург,★,5398064,Северо-Западный федеральный округ
4,Свердловская область,★,4310681,Уральский федеральный округ
...,...,...,...,...
80,Республика Алтай,,220181,Сибирский федеральный округ
81,Еврейская автономная область,,158305,Дальневосточный федеральный округ
82,Магаданская область,,140149,Дальневосточный федеральный округ
83,Чукотский автономный округ,,50288,Дальневосточный федеральный округ


### График численности населения

In [129]:
# Создаем маску, где будем использовать цвета наших регионов.
mask_number = data.groupby('Федеральный округ')['Численность населения'].sum().reset_index()
mask_number.sort_values(by='Численность населения',ascending=False,inplace=True)
mask_number['цвет'] = mask_number['Федеральный округ'].map(dct_colors)

# Нарисуем первую фигуру, которая будет состоять из гистограммы и коробчатой диаграммы
fig_1=make_subplots(rows=1,
                  cols=2,
                  subplot_titles=['Гистограмма ','Коробчатая диаграмма']
                  )
# Гистограмма
fig_1.add_trace(col=1,row=1,trace=go.Histogram(x=data['Численность населения'],showlegend=False))
fig_1.update_xaxes(title='Численность',col=1,row=1)

# Коробчатая диаграмма
fig_1.add_trace(col=2,row=1,trace=go.Box(x=data['Численность населения'],showlegend=False,name=''))
fig_1.update_xaxes(title='Численность',col=2,row=1)
# Изменим высоту и ширину первого графика
fig_1.update_layout(height=260,width=1200)

# Нарисуем первый график
fig_1.show()

# Далее нам понадобится столбчатая диаграмма, цветом выделим округа, а меткой сделаем регионы-млн.
fig_2 = px.bar(x=data['Регионы'],
             title='Численность населения по регионам',
             y=data['Численность населения'],
             color=data['Федеральный округ'],
             color_discrete_map=dct_colors,
             text=data['метка региона'],height=600,width=1200)

# Отсортируем согласно признаку Регионы, а он у нас отсортирован по численности населения
fig_2.update_xaxes(categoryorder='array',categoryarray=data['Регионы'])

# Добавим текст во внутрь черным цветом, что бы мы понимали где регион миллионник
fig_2.update_traces(textposition='outside',textfont_color='#3F51B5')


# Третий график будет внутри основного второго графика
fig_2.add_trace(
    trace=go.Bar(marker_color=mask_number['цвет'],
        x=mask_number['Федеральный округ'],
        y=mask_number['Численность населения'],
        name="Федеральный округ",xaxis='x2',
        showlegend=False))

# Рисуем внутри второго графика фигуру типа rect - это прямоугольник, указываем его вершины
fig_2.update_layout( shapes=[
    dict(type="rect", #тип
         xref="paper", # цвет 
         yref="paper", # цвет 
         x0=0.5, # начало графика x0 от левого края
         y0=0.35, # начало графика y0 от нижнего края
         x1=0.9, # конец графика x1 от левого края
         y1=0.83, # конец графика y1 от нижнего края
         line_width=2, #
         fillcolor=None, #
         opacity=1)])

# Уточняем область отображения второго графика
fig_2.update_traces(selector=lambda t: t.name=="Федеральный округ", xaxis="x2", yaxis="y2")

# Новые оси третьего графика
fig_2.update_layout(
    xaxis2=dict(domain=[0.51, 0.88], anchor="y2"),  # определяем оси X для врезанного графика
    yaxis2=dict(domain=[0.37, 0.81], anchor="x2")) # определяем оси Y для врезанного графика

fig_2.update_layout(autosize=False,
                  legend_orientation="h",
                  legend=dict(x=0,y=1.2, yanchor='top',title=''),
                  title="Численность населения",
                  xaxis_title="Регионы",
                  yaxis_title="Численность",
                  
                  xaxis2 = {'showticklabels': False},
                  yaxis2 = {'showticklabels': False})



fig_2.show()

По данному признаку можно увидеть, что:
1) Численность регионов в основном до 2х млн. человек.
2) Есть выбросы такие как Москва-12млн., МО-7.6 млн.,Краснодарский край-5.6 млн. и Санкт-Петербург-5.4млн. человек. Так и есть в    действительности это самые густонаселенные регионы нашей страны.
3) Центральный федеральный округ самый крупный, т.к. там есть Москва и МО. 
4) Самый большой регион Москва, самый маленький регион по численности это Ненецкий АО. 

## 2) Новорожденные

In [130]:
# рождённые в этом месяце, по регионам, без учёта мертворождённых. Расчет взят на 15 июня каждого года.
data_newborn_2006_2022_monthly = pd.read_csv('./social_russia_data/newborn_2006_2022_monthly.csv',sep=';')
n = [0]

for num,value in enumerate(data_newborn_2006_2022_monthly.columns):
    lst = re.findall(r'\d{4}', value) 
    if len(lst) != 0: 
        if int(lst[0]) == 2020:
            n.append(num)
           
        
data_newborn_2020 = data_newborn_2006_2022_monthly.iloc[:,n]
data_newborn_2020 = data_newborn_2020.rename(columns={'Region':'region'})

for i in data_newborn_2020.columns[1:]:
    data_newborn_2020[i] = data_newborn_2020[i].str.replace(',','.').astype('float')
mask = data_newborn_2020[data_newborn_2020.columns[1:]].sum(axis=1)
    
data_newborn_2020 = pd.DataFrame({'region':data_newborn_2020['region'],'Новорожденные':np.round(mask)})

data_newborn_2020.drop([23,71],axis=0,inplace=True)

data_newborn_2020['region'] = data_newborn_2020['region'].apply(lambda x :dct_city[x] if x in dct_city else x)
data_newborn_2020


,region,Новорожденные
0,Российская Федерация,1435750.0
1,Центральный федеральный округ,354092.0
2,Белгородская область,12352.0
3,Брянская область,9480.0
4,Владимирская область,10120.0
...,...,...
104,Еврейская автономная область,1605.0
105,Чукотский автономный округ,537.0
106,Крымский федеральный округ,0.0
107,Московская обл. в старых границах,0.0


In [131]:
data = data.merge(right=data_newborn_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

,Регионы,метка региона,Численность населения,Федеральный округ,Новорожденные
0,Москва,★,12678079,Центральный федеральный округ,123524.0
1,Московская область,★,7690863,Центральный федеральный округ,79107.0
2,Краснодарский край,★,5675462,Южный федеральный округ (с 29.07.2016),59235.0
3,Санкт-Петербург,★,5398064,Северо-Западный федеральный округ,55439.0
4,Свердловская область,★,4310681,Уральский федеральный округ,43991.0
...,...,...,...,...,...
80,Республика Алтай,,220181,Сибирский федеральный округ,2955.0
81,Еврейская автономная область,,158305,Дальневосточный федеральный округ,1605.0
82,Магаданская область,,140149,Дальневосточный федеральный округ,1314.0
83,Чукотский автономный округ,,50288,Дальневосточный федеральный округ,537.0


Необходимо уйти от выбросов в признаках и сделать данные по новорожденным более статистически, а именно, лучше оттолкнуться от процентов.

In [132]:
data['% прироста населения'] = (data['Новорожденные']*100/data['Численность населения']).astype("float16")
data

,Регионы,метка региона,Численность населения,Федеральный округ,Новорожденные,% прироста населения
0,Москва,★,12678079,Центральный федеральный округ,123524.0,0.974121
1,Московская область,★,7690863,Центральный федеральный округ,79107.0,1.028320
2,Краснодарский край,★,5675462,Южный федеральный округ (с 29.07.2016),59235.0,1.043945
3,Санкт-Петербург,★,5398064,Северо-Западный федеральный округ,55439.0,1.027344
4,Свердловская область,★,4310681,Уральский федеральный округ,43991.0,1.020508
...,...,...,...,...,...,...
80,Республика Алтай,,220181,Сибирский федеральный округ,2955.0,1.341797
81,Еврейская автономная область,,158305,Дальневосточный федеральный округ,1605.0,1.013672
82,Магаданская область,,140149,Дальневосточный федеральный округ,1314.0,0.937500
83,Чукотский автономный округ,,50288,Дальневосточный федеральный округ,537.0,1.067383


### График Новорожденные 

In [133]:
# Создаем маску для формирования нужного нам датасета.
mask_newborn = data.sort_values(by='Новорожденные',ascending=False)
mask_newborn['ОКР'] = mask_newborn['Новорожденные']*100/mask_newborn['Численность населения']
mask_newborn_OKR = mask_newborn.sort_values(by='ОКР',ascending=False)


fig_1 = px.bar(x=mask_newborn['Регионы'],
               title='Рождаемость в России',
               y=mask_newborn['Новорожденные'],
               color=mask_newborn['Федеральный округ'],
               color_discrete_map=dct_colors,
               text=mask_newborn['метка региона'],
               width=1200,height=600,range_y=[0,150000])

# Отсортируем согласно признаку Регионы, а он у нас отсортирован по численности населения
fig_1.update_xaxes(categoryorder='array',categoryarray=mask_newborn['Регионы'])

fig_1.update_traces(textposition='outside',textfont_color='#3F51B5')

fig_1.update_layout(legend_orientation='h',legend=dict(x=0, y=1.2,yanchor='top',title=''),autosize=False)

# Добавляем вставленный график
fig_1.add_trace(go.Box(x=mask_newborn['Новорожденные'],name="",showlegend=False,xaxis="x2",yaxis="y2"))


# # Конфигурация осей для вторичного графика

fig_1.update_layout(shapes=[dict(type="rect", #тип
                           xref="paper", # цвет 
                           yref="paper", # цвет 
                           x0=0.5, # начало графика x0 от левого края
                           y0=0.4, # начало графика y0 от нижнего края
                           x1=0.9, # конец графика x1 от левого края
                           y1=0.8, # конец графика y1 от нижнего края
                           line_width=2, #
                           fillcolor=None, #
                           opacity=1)], # прозрачность
)

# Уточняем область отображения второго графика
fig_1.update_traces(selector=lambda t: t.name=="", xaxis="x2", yaxis="y2")

fig_1.update_layout(
     xaxis2=dict(domain=[0.5, 0.9], anchor="y2"),  # определяем оси X для врезанного графика
     yaxis2=dict(domain=[0.4, 0.8], anchor="x2"))   # определяем оси Y для врезанного графика

fig_1.update_xaxes(title='Регионы')
fig_1.update_yaxes(title='Новорожденные')

fig_1.update_layout(
    xaxis2_title="",  # Убираем подпись оси X
    yaxis2_title=""   # Убираем подпись оси Y
)

fig_1.show()

Ситуация с рождаемостью почти такая же как и с численностью населения, т.к. чем больше регион - тем больше рождаемость. Но если взглянуть в процентом соотношении к населению региона, то ситуация совсем другая.

In [134]:
mask_perc = data.groupby('Федеральный округ')[['Численность населения','Новорожденные']].sum().reset_index()
mask_perc['Новорожденные в %'] = mask_perc['Новорожденные']*100/mask_perc['Численность населения'].astype('float')
mask_perc.sort_values(by='Новорожденные в %',ascending=False,inplace=True)
mask_perc['Новорожденные в %'] = mask_perc['Новорожденные в %'].round(2)
mask_perc['цвет'] = mask_perc['Федеральный округ'].map(dct_colors)

fig = px.bar(x=mask_newborn_OKR['Регионы'],title='Прирост населения в процентах по регионам',
             y=mask_newborn_OKR['ОКР'],
             color=mask_newborn_OKR['Федеральный округ'],
             color_discrete_map=dct_colors,
             text=mask_newborn_OKR['метка региона'],height=600,width=1200,range_y=[0,2.2])
fig.update_traces(textposition='outside',textfont_color='#3F51B5')
fig.update_xaxes(categoryorder='array',categoryarray=mask_newborn_OKR['Регионы'],title='Регионы')

fig.update_layout(autosize=False,
                  legend_orientation="h",
                  legend=dict(x=0,y=1.2, yanchor='top',title=''),
                  title="Численность населения",
                  xaxis_title="Регионы",
                  yaxis_title="Численность",
                  margin=dict(l=0, r=0, t=110, b=250))




# Второй график будет внутри основного второго графика
fig.add_trace(
    trace=go.Bar(marker_color=mask_perc['цвет'],
        x=mask_perc['Федеральный округ'],
        y=mask_perc['Новорожденные в %'],
        name="Федеральный округ",xaxis='x2',
        showlegend=False))

# Рисуем внутри графика фигуру типа rect - это прямоугольник, указываем его вершины
fig.update_layout( shapes=[
    dict(type="rect", #тип
         xref="paper", # цвет 
         yref="paper", # цвет 
         x0=0.5, # начало графика x0 от левого края
         y0=0.54, # начало графика y0 от нижнего края
         x1=0.9, # конец графика x1 от левого края
         y1=0.9, # конец графика y1 от нижнего края
         line_width=2, #
         fillcolor=None, #
         opacity=1)])

# Уточняем область отображения второго графика
fig.update_traces(selector=lambda t: t.name=="Федеральный округ", xaxis="x2", yaxis="y2")

# Новые оси второго графика
fig.update_layout(
    xaxis2=dict(domain=[0.52, 0.88], anchor="y2"),  # определяем оси X для врезанного графика
    yaxis2=dict(domain=[0.57, 0.88], anchor="x2")) # определяем оси Y для врезанного графика

fig.update_layout(autosize=False,
                  legend_orientation="h",
                  legend=dict(x=0,y=1.2, yanchor='top',title=''),
                  title="Отношение числа новорожденных к численности населения",
                  xaxis_title="Регионы",
                  yaxis_title="Численность",
                  margin=dict(l=0, r=0, t=110, b=250),
                  xaxis2 = {'showticklabels': False},
                  yaxis2 = {'showticklabels': False})

fig.show()

На графике четко видно, что регионы с большой численностью населения в чистом виде конечно дают больше прирост, но если брать в процентном, то общая картина такова, что не более 2.1% прирост населения. И то данный показатель виден только в Чеченской республике и республика Тыва, у которой численность населения составляет 328 тыс. человек. Ну и брать если округа, то тут на первом месте мы видим Северо-Кавказкий округ, далее идет Дальневосточный, Уральский, Сибирский. Ну а потом идет центральная Россия. На Кавказе традиционно высокая рождаемость, т.к. это восточное население со своими устоями, детей в семье не меньше 3-4 и более. А вот Дальневосточный, Уральский и Сибирский округа показывают рост из-за высокой поддержки государства.

По рождаемости картина почти такая же как и по численности. Выбросов чуть больше, но по итогу лидеры остались те же.

## Умершие дети на первом году жизни в городе и сельской местности 

In [135]:
#число умерших на первом году жизни детей за год, по всем регионам, в сельской местности. 

data_child_mortality_rural_1990_2021 = pd.read_excel('./social_russia_data/child_mortality_rural_1990_2021.xls')


data_child_mortality_rural_2020 = data_child_mortality_rural_1990_2021.iloc[2:,[0,33]]
data_child_mortality_rural_2020.columns = ['region','число умерших детей на первом году жизни в деревне 2020']
data_child_mortality_rural_2020['region'] = data_child_mortality_rural_2020['region'].apply(lambda x : x.strip())
data_child_mortality_rural_2020 = data_child_mortality_rural_2020.reset_index(drop=True)
data_child_mortality_rural_2020


,region,число умерших детей на первом году жизни в деревне 2020
0,Российская Федерация,1916.0
1,Центральный федеральный округ,228.0
2,Белгородская область,22.0
3,Брянская область,12.0
4,Владимирская область,10.0
...,...,...
114,Северо-Кавказский район,NaN
115,Уральский район,NaN
116,Западно-Сибирский район,NaN
117,Восточно-Сибирский район,NaN


In [136]:
data_child_mortality_rural_2020.drop([23,71],axis=0,inplace=True)

In [137]:
data_child_mortality_rural_2020['region'] = data_child_mortality_rural_2020['region'].apply(lambda x :dct_city[x] if x in dct_city else x)

In [138]:
data = data.merge(right=data_child_mortality_rural_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data 

,Регионы,метка региона,Численность населения,Федеральный округ,Новорожденные,% прироста населения,число умерших детей на первом году жизни в деревне 2020
0,Москва,★,12678079,Центральный федеральный округ,123524.0,0.974121,10.0
1,Московская область,★,7690863,Центральный федеральный округ,79107.0,1.028320,26.0
2,Краснодарский край,★,5675462,Южный федеральный округ (с 29.07.2016),59235.0,1.043945,91.0
3,Санкт-Петербург,★,5398064,Северо-Западный федеральный округ,55439.0,1.027344,0.0
4,Свердловская область,★,4310681,Уральский федеральный округ,43991.0,1.020508,38.0
...,...,...,...,...,...,...,...
80,Республика Алтай,,220181,Сибирский федеральный округ,2955.0,1.341797,21.0
81,Еврейская автономная область,,158305,Дальневосточный федеральный округ,1605.0,1.013672,5.0
82,Магаданская область,,140149,Дальневосточный федеральный округ,1314.0,0.937500,0.0
83,Чукотский автономный округ,,50288,Дальневосточный федеральный округ,537.0,1.067383,1.0


In [139]:
# число умерших на первом году жизни детей за год, по всем регионам, в городской местности

data_child_mortality_urban_1990_2021 = pd.read_excel('./social_russia_data/child_mortality_urban_1990_2021.xls')
data_child_mortality_urban_2020 = data_child_mortality_urban_1990_2021.iloc[2:,[0,33]]
data_child_mortality_urban_2020.columns = ['region','число умерших детей на первом году жизни в городе 2020']
data_child_mortality_urban_2020['region'] = data_child_mortality_urban_2020['region'].apply(lambda x: x.strip())
data_child_mortality_urban_2020 = data_child_mortality_urban_2020.reset_index().drop(['index'],axis=1)

In [140]:
data_child_mortality_urban_2020.drop([23,71],axis=0,inplace=True)

In [141]:
data_child_mortality_urban_2020['region'] = data_child_mortality_urban_2020['region'].apply(lambda x :dct_city[x] if x in dct_city else x)

In [142]:
data = data.merge(right=data_child_mortality_urban_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

,Регионы,метка региона,Численность населения,Федеральный округ,Новорожденные,% прироста населения,число умерших детей на первом году жизни в деревне 2020,число умерших детей на первом году жизни в городе 2020
0,Москва,★,12678079,Центральный федеральный округ,123524.0,0.974121,10.0,440.0
1,Московская область,★,7690863,Центральный федеральный округ,79107.0,1.028320,26.0,220.0
2,Краснодарский край,★,5675462,Южный федеральный округ (с 29.07.2016),59235.0,1.043945,91.0,139.0
3,Санкт-Петербург,★,5398064,Северо-Западный федеральный округ,55439.0,1.027344,0.0,191.0
4,Свердловская область,★,4310681,Уральский федеральный округ,43991.0,1.020508,38.0,144.0
...,...,...,...,...,...,...,...,...
80,Республика Алтай,,220181,Сибирский федеральный округ,2955.0,1.341797,21.0,1.0
81,Еврейская автономная область,,158305,Дальневосточный федеральный округ,1605.0,1.013672,5.0,6.0
82,Магаданская область,,140149,Дальневосточный федеральный округ,1314.0,0.937500,0.0,4.0
83,Чукотский автономный округ,,50288,Дальневосточный федеральный округ,537.0,1.067383,1.0,7.0


In [143]:
data['% умерших детей в деревне'] = ((data['число умерших детей на первом году жизни в деревне 2020']/data['Новорожденные'])*100).astype("float16")

data['% умерших детей в городе'] = ((data['число умерших детей на первом году жизни в городе 2020']/data['Новорожденные'])*100).astype("float16")

data

,Регионы,метка региона,Численность населения,Федеральный округ,Новорожденные,% прироста населения,число умерших детей на первом году жизни в деревне 2020,число умерших детей на первом году жизни в городе 2020,% умерших детей в деревне,% умерших детей в городе
0,Москва,★,12678079,Центральный федеральный округ,123524.0,0.974121,10.0,440.0,0.008095,0.356201
1,Московская область,★,7690863,Центральный федеральный округ,79107.0,1.028320,26.0,220.0,0.032867,0.278076
2,Краснодарский край,★,5675462,Южный федеральный округ (с 29.07.2016),59235.0,1.043945,91.0,139.0,0.153564,0.234619
3,Санкт-Петербург,★,5398064,Северо-Западный федеральный округ,55439.0,1.027344,0.0,191.0,0.000000,0.344482
4,Свердловская область,★,4310681,Уральский федеральный округ,43991.0,1.020508,38.0,144.0,0.086365,0.327393
...,...,...,...,...,...,...,...,...,...,...
80,Республика Алтай,,220181,Сибирский федеральный округ,2955.0,1.341797,21.0,1.0,0.710449,0.033844
81,Еврейская автономная область,,158305,Дальневосточный федеральный округ,1605.0,1.013672,5.0,6.0,0.311523,0.373779
82,Магаданская область,,140149,Дальневосточный федеральный округ,1314.0,0.937500,0.0,4.0,0.000000,0.304443
83,Чукотский автономный округ,,50288,Дальневосточный федеральный округ,537.0,1.067383,1.0,7.0,0.186279,1.303711


### График по умершим детям на первом году жизни 

In [144]:
mask3 = data.copy()
mask3['sum'] = mask3[
    ['число умерших детей на первом году жизни в городе 2020','число умерших детей на первом году жизни в деревне 2020']].sum(axis=1)
mask3.sort_values(by='sum',ascending=False,inplace=True)

mask3['число умерших детей на первом году жизни в городе 2020 %'] = np.round(
    mask3['число умерших детей на первом году жизни в городе 2020']*100/mask3[
        'число умерших детей на первом году жизни в городе 2020'].sum(),2)
mask3['число умерших детей на первом году жизни в деревне 2020 %'] = np.round(
    mask3['число умерших детей на первом году жизни в деревне 2020']*100/mask3[
        'число умерших детей на первом году жизни в деревне 2020'].sum(),2)

fig=make_subplots(rows=3,
                  cols=2,
                  vertical_spacing=0.008,
                  specs=[
                         [{'type':'domain','b':0.08}, 
                         {'type':'domain','b':0.08}],
                         [{"colspan": 2,'b':0.16}, None],
                         [{"colspan": 2,'t':0.16}, None]],
                  subplot_titles=['Смертность в городе','Смертность в деревне','Смертность на первом году жизни','Смертность на уровне округов'])

fig_1 = make_subplots(rows=1,cols=2,
                      subplot_titles=[
                        'число умерших детей на первом году жизни в городе 2020','число умерших детей на первом году жизни в деревне 2020'],
                      specs=[[{"r":0.15},{"r":0.15}]])

fig_1.update_layout(height=300, # общаая высота графика
                    width=1200,
                    title="Распределение смертности на первом году жизни")

fig_1.add_box(x=data['число умерших детей на первом году жизни в городе 2020'],row=1,col=1,name='город')
fig_1.add_box(x=data['число умерших детей на первом году жизни в деревне 2020'],row=1,col=2,name='деревня')

fig_1.show()

fig.add_trace(go.Bar(x=mask3['Регионы'],
                     y=mask3['число умерших детей на первом году жизни в городе 2020'],
                     name='Город',
                     showlegend=True,
                     legend="legend2"),
                     col=1,
                     row=2)

fig.add_trace(go.Bar(x=mask3['Регионы'],
                     y=mask3['число умерших детей на первом году жизни в деревне 2020'],
                     name='Деревня',
                     showlegend=True,
                     legend="legend2"),
                     col=1,
                     row=2)
                    


fig.add_trace(go.Pie(labels=mask3['Регионы'],
                     values=mask3['число умерших детей на первом году жизни в городе 2020 %'].head(10),
                     legend="legend1"),
                     col=1,
                     row=1)

fig.add_trace(go.Pie(labels=mask3['Регионы'],
                     values=mask3['число умерших детей на первом году жизни в деревне 2020 %'].head(10),
                     name='деревня',
                     legend="legend1"),
                     col=2,
                     row=1)


for group in data['Федеральный округ'].unique():
    mask = mask3[mask3['Федеральный округ']==group]
    fig.add_trace(col=1,
                  row=3,
                  trace=go.Bar(x=mask['Регионы'],
                  y=mask['sum'],
                  showlegend=True,
                  name=group,
                  legend='legend3',
                  marker_color = dct_colors[group])  )
    

for i in range(len(mask3['Регионы'])):
    if mask3['метка региона'][i]=='★':
        fig.add_annotation(row=3,col=1,
            x=mask3['Регионы'][i],
            y=mask3['sum'][i],
            text='★',
            showarrow=False,
            font=dict(size=14,color='#3F51B5'),
            align="center",
            valign="top",
            yshift=2
        ) 

for i in range(len(mask3['Регионы'])):
    if mask3['метка региона'][i]=='★':
        fig.add_annotation(row=2,col=1,
            x=mask3['Регионы'][i],
            y=mask3['sum'][i],
            text='★',
            showarrow=False,
            font=dict(size=14,color='#3F51B5'),
            align="center",
            valign="top",
            yshift=2
        ) 




fig.update_layout(title={'text':'Смертность на первом году жизни'},
                  barmode='stack',
                  height=1200, # общаая высота графика
                  width=1200,
                  legend={"title": "Регионы",
                          "xref": "container",
                          "yref": "container",
                          "y": 0.92},
                  legend2={
                          "title": "Тип населенного пункта",
                          "xref": "container",
                          "yref": "container",
                          "y": 0.67},
                  legend3={"title": "Федеральный округ",
                          "xref": "container",
                          "yref": "container",
                          "y": 0.14})

fig.update_traces(pull=[0.1], 
                  textinfo='value',
                  row=1,
                  col=1)

fig.update_traces(pull=[0,0.1], 
                  textinfo='value',
                  row=1,
                  col=2)

fig.update_xaxes(col=1,
                 row=2,
                 title='Регионы')

fig.update_yaxes(col=1,
                 row=2,
                 title='Количество')

fig.update_xaxes(row=2,col=1,categoryorder='array', categoryarray=mask3['Регионы'])
fig.update_xaxes(row=3,col=1,categoryorder='array', categoryarray=mask3['Регионы'])
   
fig.show()


Видим, что в крупных городах конечно же преобладает смертность городская. Регионы как Башкортастан, Дагестан и Чечня, тут ситуация наоборот, смертность высокая в деревнях. При этом рождаемость в Дагестане тоже находится на высоком уровне. Нельзя не отметить, что крупные регионы как Москва, Питер, МО и т.д у нас проявляются как выбросы, от которых необходимо будет избавиться в дальнейшем.

## Число людей с инвалидностью по регионам

In [145]:
# число людей с инвалидностью по регионам, по месяцам, по возрастным группам.
data_disabled_total_by_age_2017_2022 = pd.read_csv('./social_russia_data/disabled_total_by_age_2017_2022.csv')
data_disabled_total_by_age_2017_2022['date'] = pd.to_datetime(data_disabled_total_by_age_2017_2022['date'],format='%Y-%m-%d')
dfc = data_disabled_total_by_age_2017_2022[data_disabled_total_by_age_2017_2022['date']=='2020-06-01']
data_disabled_total_by_age_2020 = dfc.groupby('region')[['total','18_30','31_40','41_50','51_60','60_']].sum().reset_index()
data_disabled_total_by_age_2020.columns = ['region','всего инвалидов','18_30_инвалид','31_40_инвалид','41_50_инвалид','51_60_инвалид','60_инвалид']

data_disabled_total_by_age_2020

,region,всего инвалидов,18_30_инвалид,31_40_инвалид,41_50_инвалид,51_60_инвалид,60_инвалид
0,Алтайский край,172645.0,7116.0,13500.0,15728.0,24569.0,111732.0
1,Амурская область,64227.0,3249.0,5238.0,6366.0,9937.0,39437.0
2,Архангельская область,85080.0,2873.0,5172.0,6999.0,11852.0,58184.0
3,Астраханская область,43566.0,2668.0,3991.0,4941.0,7947.0,24019.0
4,Белгородская область,197221.0,5052.0,9588.0,14991.0,30197.0,137393.0
...,...,...,...,...,...,...,...
90,Ярославская область,98953.0,2632.0,5095.0,7401.0,13563.0,70262.0
91,г. Байконур,1155.0,135.0,128.0,191.0,352.0,349.0
92,город Москва,991311.0,28857.0,44436.0,64527.0,135177.0,718314.0
93,город Санкт-Петербург,562262.0,12651.0,20695.0,28592.0,56731.0,443593.0


In [146]:
data_disabled_total_by_age_2020['region'] = data_disabled_total_by_age_2020['region'].apply(lambda x: dct_city[x] if x in dct_city else x) 


In [147]:
data = data.merge(right=data_disabled_total_by_age_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

,Регионы,метка региона,Численность населения,Федеральный округ,Новорожденные,% прироста населения,число умерших детей на первом году жизни в деревне 2020,число умерших детей на первом году жизни в городе 2020,% умерших детей в деревне,% умерших детей в городе,всего инвалидов,18_30_инвалид,31_40_инвалид,41_50_инвалид,51_60_инвалид,60_инвалид
0,Москва,★,12678079,Центральный федеральный округ,123524.0,0.974121,10.0,440.0,0.008095,0.356201,991311.0,28857.0,44436.0,64527.0,135177.0,718314.0
1,Московская область,★,7690863,Центральный федеральный округ,79107.0,1.028320,26.0,220.0,0.032867,0.278076,412799.0,15893.0,26311.0,35526.0,65133.0,269936.0
2,Краснодарский край,★,5675462,Южный федеральный округ (с 29.07.2016),59235.0,1.043945,91.0,139.0,0.153564,0.234619,429194.0,18213.0,28292.0,40908.0,73035.0,268746.0
3,Санкт-Петербург,★,5398064,Северо-Западный федеральный округ,55439.0,1.027344,0.0,191.0,0.000000,0.344482,562262.0,12651.0,20695.0,28592.0,56731.0,443593.0
4,Свердловская область,★,4310681,Уральский федеральный округ,43991.0,1.020508,38.0,144.0,0.086365,0.327393,271245.0,12950.0,22756.0,27746.0,40626.0,167167.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Республика Алтай,,220181,Сибирский федеральный округ,2955.0,1.341797,21.0,1.0,0.710449,0.033844,21180.0,903.0,1685.0,2134.0,3405.0,13053.0
81,Еврейская автономная область,,158305,Дальневосточный федеральный округ,1605.0,1.013672,5.0,6.0,0.311523,0.373779,12466.0,677.0,1176.0,1369.0,1782.0,7462.0
82,Магаданская область,,140149,Дальневосточный федеральный округ,1314.0,0.937500,0.0,4.0,0.000000,0.304443,5063.0,322.0,596.0,700.0,946.0,2499.0
83,Чукотский автономный округ,,50288,Дальневосточный федеральный округ,537.0,1.067383,1.0,7.0,0.186279,1.303711,1641.0,175.0,225.0,283.0,395.0,563.0


In [148]:
print('процент ивалидов от всего населения страны: ', round(data['всего инвалидов'].sum()*100/data['Численность населения'].sum(),2),'%')

процент ивалидов от всего населения страны:  7.57 %


In [149]:
data['% 18_30_инвалид'] = (data['18_30_инвалид']*100/data['Численность населения']).astype("float16")
data['% 31_40_инвалид'] = (data['31_40_инвалид']*100/data['Численность населения']).astype("float16")
data['% 41_50_инвалид'] = (data['41_50_инвалид']*100/data['Численность населения']).astype("float16")
data['% 51_60_инвалид'] = (data['51_60_инвалид']*100/data['Численность населения']).astype("float16")
data['% 60_инвалид'] = (data['60_инвалид']*100/data['Численность населения']).astype("float16")
data

,Регионы,метка региона,Численность населения,Федеральный округ,Новорожденные,% прироста населения,число умерших детей на первом году жизни в деревне 2020,число умерших детей на первом году жизни в городе 2020,% умерших детей в деревне,% умерших детей в городе,...,18_30_инвалид,31_40_инвалид,41_50_инвалид,51_60_инвалид,60_инвалид,% 18_30_инвалид,% 31_40_инвалид,% 41_50_инвалид,% 51_60_инвалид,% 60_инвалид
0,Москва,★,12678079,Центральный федеральный округ,123524.0,0.974121,10.0,440.0,0.008095,0.356201,...,28857.0,44436.0,64527.0,135177.0,718314.0,0.227661,0.350586,0.508789,1.066406,5.664062
1,Московская область,★,7690863,Центральный федеральный округ,79107.0,1.028320,26.0,220.0,0.032867,0.278076,...,15893.0,26311.0,35526.0,65133.0,269936.0,0.206665,0.342041,0.461914,0.846680,3.509766
2,Краснодарский край,★,5675462,Южный федеральный округ (с 29.07.2016),59235.0,1.043945,91.0,139.0,0.153564,0.234619,...,18213.0,28292.0,40908.0,73035.0,268746.0,0.320801,0.498535,0.720703,1.287109,4.734375
3,Санкт-Петербург,★,5398064,Северо-Западный федеральный округ,55439.0,1.027344,0.0,191.0,0.000000,0.344482,...,12651.0,20695.0,28592.0,56731.0,443593.0,0.234375,0.383301,0.529785,1.050781,8.218750
4,Свердловская область,★,4310681,Уральский федеральный округ,43991.0,1.020508,38.0,144.0,0.086365,0.327393,...,12950.0,22756.0,27746.0,40626.0,167167.0,0.300537,0.527832,0.643555,0.942383,3.878906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Республика Алтай,,220181,Сибирский федеральный округ,2955.0,1.341797,21.0,1.0,0.710449,0.033844,...,903.0,1685.0,2134.0,3405.0,13053.0,0.410156,0.765137,0.969238,1.546875,5.929688
81,Еврейская автономная область,,158305,Дальневосточный федеральный округ,1605.0,1.013672,5.0,6.0,0.311523,0.373779,...,677.0,1176.0,1369.0,1782.0,7462.0,0.427734,0.742676,0.864746,1.125977,4.714844
82,Магаданская область,,140149,Дальневосточный федеральный округ,1314.0,0.937500,0.0,4.0,0.000000,0.304443,...,322.0,596.0,700.0,946.0,2499.0,0.229736,0.425293,0.499512,0.674805,1.783203
83,Чукотский автономный округ,,50288,Дальневосточный федеральный округ,537.0,1.067383,1.0,7.0,0.186279,1.303711,...,175.0,225.0,283.0,395.0,563.0,0.347900,0.447510,0.562988,0.785645,1.119141


In [150]:
ind_all = data.iloc[:,[0,1]+list(range(10,21))].sort_values(by='всего инвалидов',ascending=False)
#ind_all.iloc[:,8:] = ind_all.iloc[:,8:].round(1)

ind_18_30 = ind_all.sort_values(by='% 18_30_инвалид',ascending=False).head(10)
ind_31_40 = ind_all.sort_values(by='% 31_40_инвалид',ascending=False).head(10)
ind_41_50 = ind_all.sort_values(by='% 41_50_инвалид',ascending=False).head(10)
ind_51_60 = ind_all.sort_values(by='% 51_60_инвалид',ascending=False).head(10)
ind_60 = ind_all.sort_values(by='% 60_инвалид',ascending=False).head(10)

In [151]:
# data_mask = data.copy()

# data_mask['18_30 %'] = (data['18_30_инвалид']*100/data['18_30_инвалид'].sum()).round(2)
# data_mask['31_40 %'] = (data['31_40_инвалид']*100/data['31_40_инвалид'].sum()).round(2)
# data_mask['41_50 %'] = (data['41_50_инвалид']*100/data['41_50_инвалид'].sum()).round(2)
# data_mask['51_60 %'] = (data['51_60_инвалид']*100/data['51_60_инвалид'].sum()).round(2)
# data_mask['60_инвалид %'] = (data['60_инвалид']*100/data['60_инвалид'].sum()).round(2)

# ind_all = data_mask.sort_values(by='всего инвалидов',ascending=False)
# ind_18_30 = data_mask.sort_values(by='18_30 %',ascending=False).head(10)
# ind_31_40 = data_mask.sort_values(by='31_40 %',ascending=False).head(10)
# ind_41_50 = data_mask.sort_values(by='41_50 %',ascending=False).head(10)
# ind_51_60 = data_mask.sort_values(by='51_60 %',ascending=False).head(10)
# ind_60 = data_mask.sort_values(by='60_инвалид %',ascending=False).head(10)

### График по инвалидности

In [152]:

fig = make_subplots(2,5,
                    vertical_spacing=0.42,
                    row_heights=[0.1,0.125],
                    subplot_titles=['Инвалидность 2020','Инвалидность 18-30','Инвалидность 31-40','Инвалидность 41-50','Инвалидность 51-60',
                                    'Инвалидность 60'],
                    specs=[[{"colspan": 5,}, None, None, None, None],
                           [{'type':'domain'}, {'type':'domain'},{'type':'domain'},{'type':'domain'},{'type':'domain'}]])

fig.add_trace(row=1,
              col=1,
              trace=go.Bar(x=ind_all['Регионы'],
        y=ind_all['60_инвалид'],
        name='60_инвалид',
        legend="legend1"))
fig.add_trace(row=1,col=1,trace=go.Bar(
        x=ind_all['Регионы'],
        y=ind_all['51_60_инвалид'],
        name='51_60_инвалид',
        legend="legend1"))
fig.add_trace(row=1,
              col=1,
              trace=go.Bar(x=ind_all['Регионы'],
        y=ind_all['41_50_инвалид'],
        name='41_50_инвалид',
        legend="legend1"))
fig.add_trace(row=1,col=1,trace=go.Bar(
        x=ind_all['Регионы'],
        y=ind_all['31_40_инвалид'],
        name='31_40_инвалид',
        legend="legend1"))
fig.add_trace(row=1,col=1,trace=go.Bar(
        x=ind_all['Регионы'],
        y=ind_all['18_30_инвалид'],
        name='18_30_инвалид',
        legend="legend1"))


for i in range(len(ind_all['Регионы'])):
    if ind_all['метка региона'][i]=='★':
        fig.add_annotation(row=1,col=1,
            x=ind_all['Регионы'][i],
            y=ind_all['всего инвалидов'][i],
            text='★',
            showarrow=False,
            font=dict(size=14,color='#3F51B5'),
            align="center",
            valign="top",
            yshift=2
        )


fig.add_trace(row=2,col=1,
              trace=go.Pie(legend="legend2",
                           hovertemplate='%{label}',
                           texttemplate='%{value:.1f}%',
                           textinfo='value',
                  name='Инвалидность 18-30',
                  labels=ind_18_30['Регионы'],
                  values=ind_18_30['% 18_30_инвалид']))

fig.add_trace(row=2,col=2,
              trace=go.Pie(legend="legend2",
                           hovertemplate='%{label}',
                           texttemplate='%{value:.1f}%',
                           textinfo='value',
                  name='Инвалидность 31-40',
                  labels=ind_31_40['Регионы'],
                  values=ind_31_40['% 31_40_инвалид']))
                  
fig.add_trace(row=2,col=3,
              trace=go.Pie(legend="legend2",
                           hovertemplate='%{label}',
                           texttemplate='%{value:.1f}%',
                           textinfo='value',
                  name='Инвалидность 41-50',
                  labels=ind_41_50['Регионы'],
                  values=ind_41_50['% 41_50_инвалид']))

fig.add_trace(row=2,col=4,
              trace=go.Pie(legend="legend2",
                           hovertemplate='%{label}',
                           texttemplate='%{value:.1f}%',
                           textinfo='value',
                  name='Инвалидность 51-60',
                  labels=ind_51_60['Регионы'],
                  values=ind_51_60['% 51_60_инвалид']))

fig.add_trace(row=2,col=5,
              trace=go.Pie(legend="legend2",
                           hovertemplate='%{label}',
                           texttemplate='%{value:.1f}%',
                           textinfo='value',
                  name='Инвалидность 60',
                  labels=ind_60['Регионы'],
                  values=ind_60['% 60_инвалид']))


fig.update_layout(barmode='stack',
                  title="Инвалидность по возрастным группам",
                  height=800,
                  width=1200,
                  legend={
                          "title": "Возрастные группы",
                          "xref": "container",
                          "yref": "container",
                          "y": 0.9},
                  legend2={
                          "title": "Регионы",
                          "xref": "container",
                          "yref": "container",
                          "y": 0.1})                          
fig.update_xaxes(title='Регионы')
fig.update_yaxes(title='Численность')




fig.show()



По графикам видно, что не смотря на то, что в абсолютных числах количество инвалидов больше всего в регионах миллиониках, где численность населения высокая, то если посмотреть на графики в процентом соотношении, то тут ситуация совсем другая. На первых местах находятся регионы из Северо-Кавказского федерального округа, Чеченская Республика, Ингушетия и Дагестан. Это что касается населения до пенсионного возраста.

### Процент людей живущих за чертой бедности

In [153]:
#процент людей, живущих за чертой бедности (с денежными доходами ниже величины прожиточного минимума), оценка за год по регионам

data_poverty_percent_by_regions_1992_2020 = pd.read_csv('./social_russia_data/poverty_percent_by_regions_1992_2020.csv')
data_poverty_percent_by_regions_1992_2020[data_poverty_percent_by_regions_1992_2020['year']==2020].iloc[:,[0,2]]
data_poverty_percent_by_regions_2020 = data_poverty_percent_by_regions_1992_2020[
    data_poverty_percent_by_regions_1992_2020['year']==2020].iloc[:,[0,2]]

data_poverty_percent_by_regions_2020 = data_poverty_percent_by_regions_2020.rename(columns={'poverty_percent':'за чертой бедности в %'})
data_poverty_percent_by_regions_2020['region'] = data_poverty_percent_by_regions_2020['region'].apply(lambda x: x.strip())
data_poverty_percent_by_regions_2020


,region,за чертой бедности в %
28,Российская Федерация,12.1
70,Белгородская область,7.2
96,Брянская область,13.6
122,Владимирская область,12.5
148,Воронежская область,8.5
...,...,...
2233,Амурская область,15.2
2259,Магаданская область,8.7
2285,Сахалинская область,7.7
2311,Еврейская автономная область,23.6


In [154]:
data_poverty_percent_by_regions_2020.drop([606,1672],axis=0,inplace=True)

In [155]:
data_poverty_percent_by_regions_2020['region'] = data_poverty_percent_by_regions_2020['region'].apply(lambda x: dct_city[x] if x in dct_city else x) 
data_poverty_percent_by_regions_2020

,region,за чертой бедности в %
28,Российская Федерация,12.1
70,Белгородская область,7.2
96,Брянская область,13.6
122,Владимирская область,12.5
148,Воронежская область,8.5
...,...,...
2233,Амурская область,15.2
2259,Магаданская область,8.7
2285,Сахалинская область,7.7
2311,Еврейская автономная область,23.6


In [156]:
data = data.merge(right=data_poverty_percent_by_regions_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)

data['Население за чертой бедности'] = np.ceil(data['Численность населения']*data['за чертой бедности в %']/100)

data

,Регионы,метка региона,Численность населения,Федеральный округ,Новорожденные,% прироста населения,число умерших детей на первом году жизни в деревне 2020,число умерших детей на первом году жизни в городе 2020,% умерших детей в деревне,% умерших детей в городе,...,41_50_инвалид,51_60_инвалид,60_инвалид,% 18_30_инвалид,% 31_40_инвалид,% 41_50_инвалид,% 51_60_инвалид,% 60_инвалид,за чертой бедности в %,Население за чертой бедности
0,Москва,★,12678079,Центральный федеральный округ,123524.0,0.974121,10.0,440.0,0.008095,0.356201,...,64527.0,135177.0,718314.0,0.227661,0.350586,0.508789,1.066406,5.664062,6.0,760685
1,Московская область,★,7690863,Центральный федеральный округ,79107.0,1.028320,26.0,220.0,0.032867,0.278076,...,35526.0,65133.0,269936.0,0.206665,0.342041,0.461914,0.846680,3.509766,6.8,522979
2,Краснодарский край,★,5675462,Южный федеральный округ (с 29.07.2016),59235.0,1.043945,91.0,139.0,0.153564,0.234619,...,40908.0,73035.0,268746.0,0.320801,0.498535,0.720703,1.287109,4.734375,10.4,590249
3,Санкт-Петербург,★,5398064,Северо-Западный федеральный округ,55439.0,1.027344,0.0,191.0,0.000000,0.344482,...,28592.0,56731.0,443593.0,0.234375,0.383301,0.529785,1.050781,8.218750,5.9,318486
4,Свердловская область,★,4310681,Уральский федеральный округ,43991.0,1.020508,38.0,144.0,0.086365,0.327393,...,27746.0,40626.0,167167.0,0.300537,0.527832,0.643555,0.942383,3.878906,9.0,387962
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Республика Алтай,,220181,Сибирский федеральный округ,2955.0,1.341797,21.0,1.0,0.710449,0.033844,...,2134.0,3405.0,13053.0,0.410156,0.765137,0.969238,1.546875,5.929688,23.8,52404
81,Еврейская автономная область,,158305,Дальневосточный федеральный округ,1605.0,1.013672,5.0,6.0,0.311523,0.373779,...,1369.0,1782.0,7462.0,0.427734,0.742676,0.864746,1.125977,4.714844,23.6,37360
82,Магаданская область,,140149,Дальневосточный федеральный округ,1314.0,0.937500,0.0,4.0,0.000000,0.304443,...,700.0,946.0,2499.0,0.229736,0.425293,0.499512,0.674805,1.783203,8.7,12193
83,Чукотский автономный округ,,50288,Дальневосточный федеральный округ,537.0,1.067383,1.0,7.0,0.186279,1.303711,...,283.0,395.0,563.0,0.347900,0.447510,0.562988,0.785645,1.119141,8.0,4024


### График по населению за чертой бедности

In [157]:
# Создаем главную фигуру
med = data['за чертой бедности в %'].median()
fig = make_subplots(rows=2,cols=1,subplot_titles=['за чертой бедности в процентах','За чертой бедности в количестве жителей'],
                    specs=[[{"t":0.2}],[{"t":0.2}]])

mask_below_the_line_p =  data.sort_values(by='за чертой бедности в %',ascending=False)
mask_below_the_line_p.reset_index(drop=True,inplace=True)


# Основной график
for group in mask_below_the_line_p['Федеральный округ'].unique():
    mask = mask_below_the_line_p[mask_below_the_line_p['Федеральный округ']==group]
    fig.add_trace(trace=go.Bar(x=mask['Регионы'],
                     y=mask['за чертой бедности в %'],
                     showlegend=True,
                     name=group,
                     marker_color = dct_colors[group]),col=1,row=1)



for i in range(len(mask_below_the_line_p['Регионы'])):
    if mask_below_the_line_p['метка региона'][i]=='★':
        fig.add_annotation(col=1,row=1,
            x=mask_below_the_line_p['Регионы'][i],
            y=mask_below_the_line_p['за чертой бедности в %'][i],
            text='★',
            showarrow=False,
            font=dict(size=14,color='#3F51B5'),
            align="center",
            valign="top",
            yshift=2
        )
# Нанесем медиану на график, где указано в процентах 
fig.add_shape(
    type="line",
    x0=0, # Начало линии по X
    y0=med,
    x1=len(mask_below_the_line_p), # Конец линии по X
    y1=med,
    line=dict(color="black", width=2),
)      
fig.update_xaxes(categoryorder='array',row=1,col=1,
                  categoryarray=mask_below_the_line_p['Регионы'])
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

mask_below_the_line= data.sort_values(by='Население за чертой бедности',ascending=False)
mask_below_the_line.reset_index(drop=True,inplace=True)

for group in mask_below_the_line['Федеральный округ'].unique():
    mask = mask_below_the_line[mask_below_the_line['Федеральный округ']==group]
    fig.add_trace(trace=go.Bar(x=mask['Регионы'],
                     y=mask['Население за чертой бедности'],
                     showlegend=False,
                     name=group,
                     marker_color = dct_colors[group]),col=1,row=2)


for i in range(len(mask_below_the_line['Регионы'])):
    if mask_below_the_line['метка региона'][i]=='★':
        fig.add_annotation(col=1,row=2,
            x=mask_below_the_line['Регионы'][i],
            y=mask_below_the_line['Население за чертой бедности'][i],
            text='★',
            showarrow=False,
            font=dict(size=14,color='#3F51B5'),
            align="center",
            valign="top",
            yshift=2
        )


fig.update_layout(height=900,
                  title='Население за чертой бедности',
                  width=1200,
                  legend_orientation="h",
                  legend=dict(x=0, y=1,yanchor='top'))
                 
fig.update_legends(title='Федеральный округ')


fig.update_xaxes(categoryorder='array',row=2,col=1,
                  categoryarray=mask_below_the_line['Регионы'])
fig.show()

In [158]:
px.box(x=data['Население за чертой бедности'],width=1200,height=250,title='Распределение населения за чертой бедности')

In [159]:
# распределение малоимущего населения

poverty_socdem_2017 = pd.read_excel('./social_russia_data/poverty_socdem_2017.xls',header=2,usecols=[0,2,3,4])
poverty_socdem_2017.rename(columns={'Unnamed: 0':'region'},inplace=True)
poverty_socdem_2017['region'] = poverty_socdem_2017['region'].apply(lambda x: x.strip())
poverty_socdem_2017

,region,Дети в возрасте до 16 лет,Население старше трудоспособного возраста,Население трудоспособного возраста
0,Российская Федерация,39.3,6.6,54.1
1,Белгородская область,43.4,11.8,44.8
2,Брянская область,42.9,4.9,52.2
3,Владимирская область,34.8,8.6,56.6
4,Воронежская область,38.6,5.9,55.6
...,...,...,...,...
83,Амурская область,39.9,6.0,54.1
84,Магаданская область,32.2,14.8,53.0
85,Сахалинская область,36.4,6.4,57.1
86,Еврейская автономная область,36.0,9.9,54.1


## Распределение малоимущего населения по социально - демографическим группам

In [160]:
data_poverty_socdem_2021 = pd.read_excel('./social_russia_data/poverty_socdem_2021.xls',header=2,usecols=[0,2,3,4])
data_poverty_socdem_2021.columns=['region','% <16 за чертой бедности','% >60 за чертой бедности','% 16-60 за чертой бедности']
data_poverty_socdem_2021

,region,% <16 за чертой бедности,% >60 за чертой бедности,% 16-60 за чертой бедности
0,Российская Федерация,40.7,5.5,53.8
1,Центральный федеральный округ,0.0,0.0,0.0
2,Белгородская область,43.0,12.9,44.0
3,Брянская область,45.0,3.6,51.4
4,Владимирская область,36.4,4.9,58.7
...,...,...,...,...
87,Амурская область,40.5,5.8,53.7
88,Магаданская область,39.1,2.6,58.3
89,Сахалинская область,44.0,5.2,50.7
90,Еврейская автономная область,36.5,7.8,55.7


In [161]:
# распределение малоимущего населения по социально-демографическим группам (дети, трудящиеся, пенсионеры) за 2017–2020 гг., по регионам.
data_poverty_socdem_2021 = pd.read_excel('./social_russia_data/poverty_socdem_2021.xls',header=2,usecols=[0,2,3,4])
data_poverty_socdem_2021.columns=['region','% <16 за чертой бедности','% >60 за чертой бедности','% 16-60 за чертой бедности']

data_poverty_socdem_2021['region'] = data_poverty_socdem_2021['region'].apply(lambda x: x.strip())
data_poverty_socdem_2021[data_poverty_socdem_2021['% <16 за чертой бедности'].isna()]

,region,% <16 за чертой бедности,% >60 за чертой бедности,% 16-60 за чертой бедности
40,Город федерального значения Севастополь,NaN,NaN,NaN
91,Чукотский автономный округ,NaN,NaN,NaN


Нам необходимо либо удалить данные строки из датасета либо найти данные, возьмем данные за 2018г. Уверен, что картина за 2 года кардинально не изменилась.

In [162]:
data_poverty_socdem_2018 = pd.read_excel('./social_russia_data/poverty_socdem_2018.xls',header=2,usecols=[0,2,3,4])

col = ['region','% <16 за чертой бедности','% >60 за чертой бедности','% 16-60 за чертой бедности']
data_poverty_socdem_2018.columns = col

data_poverty_socdem_2018['region'] = data_poverty_socdem_2018['region'].apply(lambda x: x.strip())

mask = data_poverty_socdem_2018[data_poverty_socdem_2018['region'].isin(
    data_poverty_socdem_2021[data_poverty_socdem_2021['% <16 за чертой бедности'].isna()]['region'])]
mask

,region,% <16 за чертой бедности,% >60 за чертой бедности,% 16-60 за чертой бедности
38,Город федерального значения Севастополь,41.6,7.2,51.1
87,Чукотский автономный округ,51.0,0.0,49.0


Нам необходимо в таблице 2021 года заменить значения 40 и 91 строки.

In [163]:
data_poverty_socdem_2021.iloc[[40,91],[1,2,3]] = mask.iloc[:,[1,2,3]]

In [164]:
data_poverty_socdem_2021['region'] = data_poverty_socdem_2021['region'].apply(lambda x: dct_city[x] if x in dct_city else x) 
data_poverty_socdem_2021

,region,% <16 за чертой бедности,% >60 за чертой бедности,% 16-60 за чертой бедности
0,Российская Федерация,40.7,5.5,53.8
1,Центральный федеральный округ,0.0,0.0,0.0
2,Белгородская область,43.0,12.9,44.0
3,Брянская область,45.0,3.6,51.4
4,Владимирская область,36.4,4.9,58.7
...,...,...,...,...
87,Амурская область,40.5,5.8,53.7
88,Магаданская область,39.1,2.6,58.3
89,Сахалинская область,44.0,5.2,50.7
90,Еврейская автономная область,36.5,7.8,55.7


In [165]:
# распределение малоимущего населения по социально-демографическим группам (дети, трудящиеся, пенсионеры) за 2017–2021 гг., по регионам.

data = data.merge(right=data_poverty_socdem_2021,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

,Регионы,метка региона,Численность населения,Федеральный округ,Новорожденные,% прироста населения,число умерших детей на первом году жизни в деревне 2020,число умерших детей на первом году жизни в городе 2020,% умерших детей в деревне,% умерших детей в городе,...,% 18_30_инвалид,% 31_40_инвалид,% 41_50_инвалид,% 51_60_инвалид,% 60_инвалид,за чертой бедности в %,Население за чертой бедности,% <16 за чертой бедности,% >60 за чертой бедности,% 16-60 за чертой бедности
0,Москва,★,12678079,Центральный федеральный округ,123524.0,0.974121,10.0,440.0,0.008095,0.356201,...,0.227661,0.350586,0.508789,1.066406,5.664062,6.0,760685,28.7,15.0,56.3
1,Московская область,★,7690863,Центральный федеральный округ,79107.0,1.028320,26.0,220.0,0.032867,0.278076,...,0.206665,0.342041,0.461914,0.846680,3.509766,6.8,522979,43.4,6.6,50.0
2,Краснодарский край,★,5675462,Южный федеральный округ (с 29.07.2016),59235.0,1.043945,91.0,139.0,0.153564,0.234619,...,0.320801,0.498535,0.720703,1.287109,4.734375,10.4,590249,43.4,6.7,49.9
3,Санкт-Петербург,★,5398064,Северо-Западный федеральный округ,55439.0,1.027344,0.0,191.0,0.000000,0.344482,...,0.234375,0.383301,0.529785,1.050781,8.218750,5.9,318486,36.6,4.2,59.2
4,Свердловская область,★,4310681,Уральский федеральный округ,43991.0,1.020508,38.0,144.0,0.086365,0.327393,...,0.300537,0.527832,0.643555,0.942383,3.878906,9.0,387962,46.0,3.6,50.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Республика Алтай,,220181,Сибирский федеральный округ,2955.0,1.341797,21.0,1.0,0.710449,0.033844,...,0.410156,0.765137,0.969238,1.546875,5.929688,23.8,52404,42.9,1.8,55.3
81,Еврейская автономная область,,158305,Дальневосточный федеральный округ,1605.0,1.013672,5.0,6.0,0.311523,0.373779,...,0.427734,0.742676,0.864746,1.125977,4.714844,23.6,37360,36.5,7.8,55.7
82,Магаданская область,,140149,Дальневосточный федеральный округ,1314.0,0.937500,0.0,4.0,0.000000,0.304443,...,0.229736,0.425293,0.499512,0.674805,1.783203,8.7,12193,39.1,2.6,58.3
83,Чукотский автономный округ,,50288,Дальневосточный федеральный округ,537.0,1.067383,1.0,7.0,0.186279,1.303711,...,0.347900,0.447510,0.562988,0.785645,1.119141,8.0,4024,51.0,0.0,49.0


### График по распределению малоимущего населения по социально-демографическим группам

In [168]:
fig = go.Figure()
fig.add_trace(go.Bar(x=data['Регионы'],
                     y=data['% >60 за чертой бедности'],
                     name='Пенсионеры'))

fig.add_trace(go.Bar(x=data['Регионы'],
                     y=data['% 16-60 за чертой бедности'],
                     name='Люди трудоспособного возраста'))

fig.add_trace(go.Bar(x=data['Регионы'],
                     y=data['% <16 за чертой бедности'],
                     name='Дети'))

fig.update_layout(barmode='stack',
                  title='Распределение населения за чертой бедности')
fig.update_legends(title='Группы населения')
fig.update_xaxes(title='Регионы')
fig.update_yaxes(title='Численность в процентах')

for i in range(len(data['Регионы'])):
    if data['метка региона'][i]=='★':
        fig.add_annotation(
            x=data['Регионы'][i],
            y=100,
            text='★',
            showarrow=False,
            font=dict(size=14,color='#3F51B5'),
            align="center",
            valign="top",
            yshift=2
        )
fig.show()

Конечно, если мы оставим данные как они есть, они не будут отображать действительности, т.к. при общем проценте 6 мы видим распределение как и при 12%, поэтому лучше всего оттолкнуться от численности населения.

In [213]:
data['доля <16 за чертой бедности'] = (data['за чертой бедности в %']*data['% <16 за чертой бедности']/10000).astype("float")
data['доля 16-60 за чертой бедности'] = (data['за чертой бедности в %']*data['% 16-60 за чертой бедности']/10000).astype("float")
data['доля >60 за чертой бедности'] = (data['за чертой бедности в %']*data['% >60 за чертой бедности']/10000).astype("float")
data

,Регионы,метка региона,Численность населения,Федеральный округ,Новорожденные,% прироста населения,число умерших детей на первом году жизни в деревне 2020,число умерших детей на первом году жизни в городе 2020,% умерших детей в деревне,% умерших детей в городе,...,% 51_60_инвалид,% 60_инвалид,за чертой бедности в %,Население за чертой бедности,% <16 за чертой бедности,% >60 за чертой бедности,% 16-60 за чертой бедности,доля <16 за чертой бедности,доля 16-60 за чертой бедности,доля >60 за чертой бедности
0,Москва,★,12678079,Центральный федеральный округ,123524.0,0.974121,10.0,440.0,0.008095,0.356201,...,1.066406,5.664062,6.0,760685,28.7,15.0,56.3,0.017220,0.033780,0.009000
1,Московская область,★,7690863,Центральный федеральный округ,79107.0,1.028320,26.0,220.0,0.032867,0.278076,...,0.846680,3.509766,6.8,522979,43.4,6.6,50.0,0.029512,0.034000,0.004488
2,Краснодарский край,★,5675462,Южный федеральный округ (с 29.07.2016),59235.0,1.043945,91.0,139.0,0.153564,0.234619,...,1.287109,4.734375,10.4,590249,43.4,6.7,49.9,0.045136,0.051896,0.006968
3,Санкт-Петербург,★,5398064,Северо-Западный федеральный округ,55439.0,1.027344,0.0,191.0,0.000000,0.344482,...,1.050781,8.218750,5.9,318486,36.6,4.2,59.2,0.021594,0.034928,0.002478
4,Свердловская область,★,4310681,Уральский федеральный округ,43991.0,1.020508,38.0,144.0,0.086365,0.327393,...,0.942383,3.878906,9.0,387962,46.0,3.6,50.4,0.041400,0.045360,0.003240
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,Республика Алтай,,220181,Сибирский федеральный округ,2955.0,1.341797,21.0,1.0,0.710449,0.033844,...,1.546875,5.929688,23.8,52404,42.9,1.8,55.3,0.102102,0.131614,0.004284
81,Еврейская автономная область,,158305,Дальневосточный федеральный округ,1605.0,1.013672,5.0,6.0,0.311523,0.373779,...,1.125977,4.714844,23.6,37360,36.5,7.8,55.7,0.086140,0.131452,0.018408
82,Магаданская область,,140149,Дальневосточный федеральный округ,1314.0,0.937500,0.0,4.0,0.000000,0.304443,...,0.674805,1.783203,8.7,12193,39.1,2.6,58.3,0.034017,0.050721,0.002262
83,Чукотский автономный округ,,50288,Дальневосточный федеральный округ,537.0,1.067383,1.0,7.0,0.186279,1.303711,...,0.785645,1.119141,8.0,4024,51.0,0.0,49.0,0.040800,0.039200,0.000000


In [174]:
data.columns

Index(['Регионы', 'метка региона', 'Численность населения',
       'Федеральный округ', 'Новорожденные', '% прироста населения',
       'число умерших детей на первом году жизни в деревне 2020',
       'число умерших детей на первом году жизни в городе 2020',
       '% умерших детей в деревне', '% умерших детей в городе',
       'всего инвалидов', '18_30_инвалид', '31_40_инвалид', '41_50_инвалид',
       '51_60_инвалид', '60_инвалид', '% 18_30_инвалид', '% 31_40_инвалид',
       '% 41_50_инвалид', '% 51_60_инвалид', '% 60_инвалид',
       'за чертой бедности в %', 'Население за чертой бедности',
       '% <16 за чертой бедности', '% >60 за чертой бедности',
       '% 16-60 за чертой бедности', 'доля <16 за чертой бедности',
       'доля 16-60 за чертой бедности', 'доля >60 за чертой бедности'],
      dtype='object')

In [ ]:
mask_16 = data.sort_values(by='% <16 за чертой бедности').head(10)
fig = go.Figure(layout=dict(title='Распределение групп населения за чертой бедности',
                             legend=dict(title='группа')))


fig.update_traces(root_color="lightgrey")
fig.show()

In [179]:
mask_16

,Регионы,метка региона,Численность населения,Федеральный округ,Новорожденные,% прироста населения,число умерших детей на первом году жизни в деревне 2020,число умерших детей на первом году жизни в городе 2020,% умерших детей в деревне,% умерших детей в городе,...,% 51_60_инвалид,% 60_инвалид,за чертой бедности в %,Население за чертой бедности,% <16 за чертой бедности,% >60 за чертой бедности,% 16-60 за чертой бедности,доля <16 за чертой бедности,доля 16-60 за чертой бедности,доля >60 за чертой бедности
60,Республика Мордовия,,790197,Приволжский федеральный округ,5558.0,0.703125,8.0,9.0,0.143921,0.161987,...,1.557617,5.511719,17.6,139075,14.6,8.6,76.8,0.025696,0.135168,0.015136
53,Ивановская область,,997135,Центральный федеральный округ,7563.0,0.758301,3.0,25.0,0.039673,0.330566,...,1.139648,5.187500,13.7,136608,26.7,2.0,71.3,0.036579,0.097681,0.002740
0,Москва,★,12678079,Центральный федеральный округ,123524.0,0.974121,10.0,440.0,0.008095,0.356201,...,1.066406,5.664062,6.0,760685,28.7,15.0,56.3,0.017220,0.033780,0.009000
49,Калининградская область,★,1012512,Северо-Западный федеральный округ,9286.0,0.916992,5.0,30.0,0.053833,0.322998,...,1.023438,4.414062,13.5,136690,33.4,10.7,55.9,0.045090,0.075465,0.014445
32,Тульская область,★,1466127,Центральный федеральный округ,10812.0,0.737305,15.0,49.0,0.138794,0.453125,...,1.677734,6.296875,10.1,148079,34.3,4.7,61.0,0.034643,0.061610,0.004747
52,Калужская область,★,1002575,Центральный федеральный округ,8989.0,0.896484,8.0,20.0,0.088989,0.222534,...,1.210938,4.949219,9.7,97250,34.4,4.1,61.5,0.033368,0.059655,0.003977
47,Томская область,★,1079271,Сибирский федеральный округ,9940.0,0.920898,12.0,33.0,0.120728,0.332031,...,0.999512,3.248047,14.4,155416,34.9,1.9,63.2,0.050256,0.091008,0.002736
44,Рязанская область,★,1108847,Центральный федеральный округ,8701.0,0.784668,15.0,31.0,0.172363,0.356201,...,1.788086,8.070312,12.8,141933,35.3,5.8,58.9,0.045184,0.075392,0.007424
35,Пензенская область,★,1305563,Приволжский федеральный округ,9663.0,0.740234,18.0,25.0,0.186279,0.258789,...,1.178711,4.785156,13.0,169724,35.3,7.3,57.4,0.045890,0.074620,0.009490
33,Владимирская область,★,1358416,Центральный федеральный округ,10120.0,0.745117,10.0,51.0,0.098816,0.503906,...,1.349609,6.128906,12.5,169802,36.4,4.9,58.7,0.045500,0.073375,0.006125


In [188]:
data.columns

Index(['Регионы', 'метка региона', 'Численность населения',
       'Федеральный округ', 'Новорожденные', '% прироста населения',
       'число умерших детей на первом году жизни в деревне 2020',
       'число умерших детей на первом году жизни в городе 2020',
       '% умерших детей в деревне', '% умерших детей в городе',
       'всего инвалидов', '18_30_инвалид', '31_40_инвалид', '41_50_инвалид',
       '51_60_инвалид', '60_инвалид', '% 18_30_инвалид', '% 31_40_инвалид',
       '% 41_50_инвалид', '% 51_60_инвалид', '% 60_инвалид',
       'за чертой бедности в %', 'Население за чертой бедности',
       '% <16 за чертой бедности', '% >60 за чертой бедности',
       '% 16-60 за чертой бедности', 'доля <16 за чертой бедности',
       'доля 16-60 за чертой бедности', 'доля >60 за чертой бедности'],
      dtype='object')

In [260]:
mask_16 = data.sort_values(by='доля <16 за чертой бедности')
mask_16['доля <16 за чертой бедности'] = mask_16['доля <16 за чертой бедности']*100
# Создаем диаграмму treemap
fig = px.treemap(height=400,width=400,
    data_frame=mask_16,
    path=['Регионы'],   # путь к данным (родители и потомки)
    values='доля <16 за чертой бедности',
    color='Федеральный округ',
    color_discrete_map=dct_colors,
         # какие значения используются для размера участков
)

fig.update_traces(marker_line_width=2, marker_line_color="black")
# Показываем график
fig.update_layout(
    paper_bgcolor='LightSteelBlue',           
    font_family='Arial',                      
    font_size=20,
    title_font_size=24,
    margin=dict(l=10, r=10, t=60, b=10))
fig.update_traces(texttemplate="%{label}<br>%{value:.2s}%",)
fig.show()

## Болезни

In [ ]:
data_morbidity_2020_2021_age_disease = pd.read_excel('./social_russia_data/morbidity_2005_2020_age_disease.xls')
data_morbidity_2020_2021_age_disease = data_morbidity_2020_2021_age_disease.iloc[2:,:]
data_morbidity_2020_2021_age_disease.columns = ['region','вид болезни','возраст',2020,2021]

data_morbidity_2020_2021_age_disease['region'] = data_morbidity_2020_2021_age_disease['region'].apply(lambda x: x.strip())
data_morbidity_2020_2021_age_disease['region'] = data_morbidity_2020_2021_age_disease['region'].apply(lambda x: dct_city[x] if x in dct_city else x) 
data_morbidity_2020_2021_age_disease = data_morbidity_2020_2021_age_disease[(data_morbidity_2020_2021_age_disease['вид болезни'] !='Все заболевания')&(data_morbidity_2020_2021_age_disease['возраст'] !='Всего') ]

data_morbidity_2020_2021_age_disease  = data_morbidity_2020_2021_age_disease.groupby(by =['region','вид болезни'],as_index=False)[2020].sum().round()
data_morbidity_2020 = data_morbidity_2020_2021_age_disease.pivot_table(columns='вид болезни',index='region',values=2020).reset_index()


data_morbidity_2020.iloc[:,1:] = data_morbidity_2020.iloc[:,1:]


In [ ]:
data_morbidity_2020

In [ ]:
# data = data.merge(right=data_morbidity_2020,
#                   how='left',
#                   left_on='Регионы',
#                   right_on='region').drop(['region'],axis=1)
# data

Для дальнейшей визуализации признаков, касающиеся болезней, напишем функцию, которая будет принимать признак, а на выходе будет график.

In [ ]:
def func_fig(col):
    mask = data.sort_values(by=col,ascending=False)

    fig = go.Figure()
    for group in mask['Федеральный округ'].unique():
        msk = mask[mask['Федеральный округ']==group]
        fig.add_trace(trace=go.Bar(x=msk['Регионы'],
                     y=msk[col],
                     showlegend=True,
                     name=group,
                     marker_color = dct_colors[group]))

    for i in range(len(mask['Регионы'])):
        if mask['метка региона'][i]=='★':
            fig.add_annotation(
                x=mask['Регионы'][i],
                y=mask[col][i],
                text='★',
                showarrow=False,
                font=dict(size=14,color='#3F51B5'),
                align="center",
                valign="top",
                yshift=2
            )    



    fig.update_layout(title=col + ' на 100 тыс. человек в регионе',
                  height=550,
                  width=1200,
                  legend_orientation="h",
                  legend=dict(x=0, y=-2,yanchor='top'))
    fig.update_xaxes(title='Регионы',categoryorder='array',categoryarray=mask['Регионы'],
                 )
    fig.update_yaxes(title='Количество жителей')
    return fig.show()

### График заболеваемости населения

Итак рассмотрим добавленные признаки по отдельности!

In [ ]:
#func_fig('Беременность, роды и послеродовой период')

На графике заметно, что округа как Сибирский, Дальневосточный и Уральский лидируют в данном вопросе. Как и было замечено ранее в данных регионах прирост населения выше чем у центральной части страны. Тут не особо замечен Северо-Кавказкий федеральный округ. Возможно это связано с низким уровнем медицины, но пока это только предположение.

In [ ]:
#func_fig('Болезни глаза и его придаточного аппарата')

На первом месте по данному показателю Курганская область, засушливый климат, продолжительная зима, январь -18 -20, июль +18 +20, плюс сильные ветра. Дагестан на втором месте,учитывая, что высокий уровень смертности на первом году жизни в деревнях, тут низкий уровень медицины и преобладающее количество деревень, а не крупных городов, где медицина на более высоком уровне. А дальше Алтай,Чукотка, Ненецкий АО, это так же продолжительные зимы и климат резко-континентальный. Важно заметить, что Чеченская республика, являясь регионом миллионником на последнем месте.

In [ ]:
#func_fig('Болезни кожи и подкожной клетчатки')

Такое чувство, что Северо-Западный округ лидер по болезням кожи. Скорее всего это связано с климатом, т.к. он наиболее сырой и избыточная влага может вызывать закупоривание пор кожи, а так же в помощении при влаге образуется плесень, которая может вызвать алергию. Высокая влажность и переменчивость погоды из-за влияния Атлантического океана и частых циклонов, что приводит к избыточному увлажнению и заболоченности, несмотря на то, что годовое количество осадков не всегда велико. 

In [ ]:
#func_fig('Болезни костно-мышечной системы и соединительной ткани')

Высокий уровень заболеваемости болезнями костно-мышечной системы и соединительной ткани в Санкт-Петербурге может быть связан с увеличением распространенности заболеваний в период пандемии COVID-19, причем темпы роста в городе опережали общероссийские. Точные причины высокого уровня заболеваемости требуют дальнейших исследований, однако, как правило, такие заболевания связаны с различными факторами, включая генетическую предрасположенность, образ жизни, условия труда и окружающей среды.В Архангельской области высокий уровень может быть связан с климатическими факторами, низким уровнем инсоляции (дефицитом солнечного света), влияющим на обмен витамина D, а также с особенностями профессиональной деятельности жителей региона, в том числе связанной с тяжелым физическим трудом. Среди других причин могут быть наследственная предрасположенность, перенесенные инфекции и травмы.


In [ ]:
#func_fig('Болезни крови, кроветворных органов и отдельные нарушения, вовлекающие иммунный механизм')

Такой высокий уровень болезни крови в Северо-Кавказком округе связан с тем, что очень много браков среди родственников, т.к. эти регионы раньше были в изоляции. Особенно Дагестан. Это единственный пока график, где Чеченская республика была в лидерах

In [ ]:
#func_fig('Болезни мочеполовой системы')

Такое чувство, что в Северо-Западном федеральном округе  преобладает более старшее население, т.к. данная болезнь зачастую встречается у людей старше 40 лет. Возможно из-за климата данный показатель очень высокий, но больше похоже на первый вариант.

In [ ]:
#func_fig('Болезни нервной системы')

Первый раз в лидеры покала Хакасия. Болезни нервной системы, мигрень,инсульт,эпилепсия и т.д. Скорее всего из-за особенностей погоды и давления регионы выше попали в лидеры. Т.к. тут множество причин, утомляемость, нехватка воздуха в засушливое лето, общий стресс в крупных городах. Тут и центральная часть тоже присутствует. Болезни нервной системы так же могут быть связаны с общей социально-экономической ситуацией в регионе. Ведь определенные причины могут вызывать стрессы, финансовая часть, тяжелая работа добычи полезных ископаемых, суровый климат.

In [ ]:
#func_fig('Болезни нервной системы, болезни глаза и его придаточного аппарата, болезни уха и сосцевидного отростка')

На графике, где идет суммирование болезней, которые мы рассматривали ранее лидером остается Северо-Западный федеральный округ. 

In [ ]:
#func_fig('Болезни органов дыхания')

Данный вид болезни связан со средой. Тяжелые погодные условия, добыча полезных ископаемых, отсутствие достаточного количества солнечных лучей(витамина D), сырость. ОРВИ, бронхит,пневмония,бронхиальная астма,ринит,плеврит.Все эти болезни появляются от влажности и загрязнения окружающей среды. Тут следует заметить, что в Северо-Кавказкий федеральный округ по прежнему самый последний.

In [ ]:
#func_fig('Болезни органов пищеварения')

Стандартно Северо-Западный федеральный округ занимает лидерство, так же тут Дальневосточный федеральный округ. Ненецкий АО лишен натуральных продуктов,т.к. преобладает холодный климат. 240 дней зимы и холодное лето. Поэтому питаются в основном только консервацией. В Дагестане некачественное водоснабжение и изношенность инфраструктуры водоотведения, что приводит к водным инфекциям, а также неправильное питание.Причины высокого уровня заболеваемости болезнями органов пищеварения в Амурской области включают некачественное питание, недостаток чистой питьевой воды, экологические проблемы (особенно с выбросами предприятий), а также употребление спиртных напитков и курение как факторы риска, которые приводят к развитию хронических заболеваний органов пищеварения. Странно, что тут Пензенская область в лидерах, т.к. в основном все заболевания связаны с местоположением региона. Такой уровень можно связать с высоким потребленим фаст фуда.

In [ ]:
#func_fig('Болезни системы кровообращения')

Тут заметно прям, что данная болезнь наиболее распространена в регионах миллиониках, связано с малоподвижным образом жизни,экологией,курение, алкоголь, фаст фуд, Москва на последний местах по данному виду заболеваемости.Так же в Курганской области присутствуют очень высокие колебания температуры воздуха из-за особенности расположения региона.

In [ ]:
#func_fig('Болезни уха и сосцевидного отростка')

Ранее рассматривали этот момент. Сереро-Западный округ лидирует.

In [ ]:
#func_fig('Болезни эндокринной системы, расстройства питания и нарушения обмена веществ')

В среднем уровень почти одинаковый, но вот в суровых регионах как Ненецкий автономный округ, там продолжительность зимы 240 дней, что влияет на качество продуктов, в основном консервированные.

In [ ]:
#func_fig('Врожденные аномалии (пороки развития), деформации и хромосомные нарушения')

Т.к. речь идет о врожденных аномалий, то это уже специфика разных регионов страны. Но опять мы видим, что в Чеченской республике традиционно низкий показатель.В Севастополе может быть связан с рядом факторов, влияющих на развитие плода и формирование здоровья ребенка. Вот как не странно, Северо-Кавказкие регионы, кроме Ставропольского края тут занимают очень низкие места, возможно все это связано со здоровьем матери, которые в этом регионе не курят, не пьют алкоголь, не ведут "тусовочный" образ жизни,рожают в более ранем возрасте, все это влияет на врожденные пороки.Даже Ленинградская область по сравнению с Питером имеет очень низкий показатель. Ну и конечно во всех регионах очень сильно влияет качество окружающей среды.

In [ ]:
#func_fig('Некоторые инфекционные и паразитарные болезни')

Как я и говорил ранее, более влажный климат порождает инфекционные и паразитарные болезни. Это видно на графике. 

In [ ]:
#func_fig('Новообразования')

Такое чувство, что Северо-Западный федеральный округ лидер по всем заболеваниям. Конечно и Москва тоже близка в данном признаке к лидерству. Новообразования зависят от окружающей среды, от качества воды и водоснабжения, от качества канализационных каммуникаций.

In [ ]:
#func_fig('Отдельные состояния, возникающие в перинатальном периоде')

Отдельное состояние, возникающее в перинатальном периоде, называется перинатальной патологией. Перинатальный период охватывает промежуток времени непосредственно перед рождением, во время родов и сразу после рождения ребёнка (обычно первые семь дней).

Перинатальная патология включает в себя различные расстройства и состояния, которые могут повлиять на здоровье новорождённого. К таким состояниям относятся:

- Гипоксия (недостаток кислорода)
- Асфиксия (остановка дыхания)
- Родовые травмы
- Инфекции, полученные во время родов или вскоре после них
- Патологии, вызванные нарушением кровообращения плода
- Проблемы, связанные с преждевременными родами или задержкой внутриутробного развития

Забайкальский край отличается высоким уровнем отдельных состояний, возникающих в перинатальном периоде, что обусловлено несколькими факторами:

▌ Причины повышенного уровня перинатальных проблем

1. Климатические условия: Резко континентальный климат края с большими колебаниями температуры и влажностью воздуха оказывает негативное влияние на здоровье беременных женщин и новорожденных.

2. Социально-экономическое положение: Ограниченный доступ к качественной медицинской помощи, недостаток квалифицированных специалистов и современные технологии диагностики и лечения создают трудности в профилактике и лечении перинатальных заболеваний.

3. Экология: Загрязнение окружающей среды промышленными предприятиями и транспортом может способствовать возникновению патологических состояний у новорожденных.

4. Особенности питания: Недостаток витаминов и минералов, необходимых для нормального развития плода, также играет свою роль.

5. Генетические факторы: Высокая частота встречаемости определённых генетических болезней и инфекций, таких как туберкулёз, ВИЧ-инфекция и гепатит C, усиливает риск неблагоприятных исходов для новорождённых.

▌ Какие состояния встречаются чаще всего?

Среди основных состояний, часто наблюдаемых в Забайкальском крае, выделяются:

- Гипоксия и асфиксия новорождённых
- Внутриутробные инфекции
- Патологии сердечно-сосудистой системы
- Недоношенность и низкая масса тела при рождении
- Нарушения нервной системы и головного мозга

Чтобы снизить уровень перинатальной патологии, важно улучшать медицинское обслуживание, обеспечивать регулярное наблюдение за беременными женщинами, развивать профилактику инфекционных заболеваний и повышать доступность современных методов диагностики и терапии.

In [ ]:
#func_fig('Психические расстройства и расстройства поведения')

Причины опять одни и те же, окружающая среда, уровень медицины, уровень зарплат, соц. поддержка и т.д.

In [ ]:
#func_fig('Симптомы,признаки и отклонения,выявленные при клинических и лабораторных исследованиях')

Данный признак нам придется удалить из-за присутствия огромного количества нулей.

In [ ]:
#func_fig('Травмы, отравления и другие последствия воздействия внешних причин')

Уровень более менее одинаковый, внешние причины часто влияют на качество жизни людей, Погода, загрязнение окружающей среды, полезные ископаемые, влажность воздуха и т.д.

Итак есть несколько наблюдений, которые можно подчеркнуть для дальнейшего анализа.
1) По графикам выше высокий уровень болезней разных типов в Северо-Западном федеральном округе.
2) Наименьший уровень это в Северо-Кавказком регионе.
3) Москва и МО оказались средничком.
4) Зачастую  уровень какой-либо болезни связан с окружающей средой, расположением региона, климатом, особенностью региона, уровнем жизни и уровнем медицины. 

## Расходы на социальную политику

In [ ]:
# расходы на социальную политику от общих расходов бюджета региона, % в год*

data_welfare_expense_share_2015_2020 = pd.read_excel('./social_russia_data/welfare_expense_share_2015_2020.xlsx')
data_welfare_expense_share_2020 = data_welfare_expense_share_2015_2020.loc[:,['region',2020]]
data_welfare_expense_share_2020 = data_welfare_expense_share_2020.rename(columns={2020:'расходы на соц.пол. от бюджета %'})
# Этот файл нам нужен будет для аналитики 2020 г.
data_welfare_expense_share_2020['region'] = data_welfare_expense_share_2020['region'].apply(lambda x: x.strip().replace('\n',' '))

data_welfare_expense_share_2020

In [ ]:
#data_budget_expenses_2020 = pd.read_excel('./social_russia_data/data_budget_expenses_2020_145_v20250327.xlsx')


In [ ]:
# data_budget_expenses_2020_copy = data_budget_expenses_2020.copy()

# data_budget_expenses_2020_copy = data_budget_expenses_2020_copy[data_budget_expenses_2020_copy['month']==12]
# cols_expenses_2020 = data_budget_expenses_2020_copy.columns[[2,4,8,9]]
# data_budget_expenses_2020_copy = data_budget_expenses_2020_copy[cols_expenses_2020]
# data_budget_expenses_2020_copy = data_budget_expenses_2020_copy[(data_budget_expenses_2020_copy['expense_part']=='Расходы бюджета - ИТОГО') |
#     (data_budget_expenses_2020_copy['expense_part']=='СОЦИАЛЬНАЯ ПОЛИТИКА')]

# data_budget_expenses_2020_copy  = data_budget_expenses_2020_copy[
#     (data_budget_expenses_2020_copy['indicator_name'] == 'Исполнено на первое число следующего месяца, конс. бюджет субъекта РФ') &
#      (data_budget_expenses_2020_copy['expense_part'] == 'СОЦИАЛЬНАЯ ПОЛИТИКА')].iloc[:,[0,3]]
# data_budget_expenses_2020_copy.columns =['region','расходы на соц. политику']
# data_budget_expenses_2020_copy

In [ ]:
data_welfare_expense_share_2020['region'] = data_welfare_expense_share_2020['region'].apply(lambda x: dct_city[x] if x in dct_city else x) 
data_welfare_expense_share_2020

In [ ]:
set(data['Регионы']).difference(set(data_welfare_expense_share_2020['region']))

In [ ]:
data = data.merge(right=data_welfare_expense_share_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

In [ ]:
# data_budget_expenses_2020_copy['region'] = data_budget_expenses_2020_copy['region'].apply(lambda x: dct_city[x] if x in dct_city else x) 
# data_budget_expenses_2020_copy

In [ ]:
# data = data.merge(right=data_budget_expenses_2020_copy,
#                   how='left',
#                   left_on='Регионы',
#                   right_on='region').drop(['region'],axis=1)


In [ ]:
# data['расходы на соц. политику'] = np.floor(data['расходы на соц. политику'])
# data

In [ ]:
data

## График расходов на социальную политику

In [ ]:
#mask_budget = data.sort_values(by='расходы на соц. политику',ascending=False).reset_index(drop=True)
#mask_budget_perc = data.sort_values(by='расходы на соц.пол. от бюджета %',ascending=False).reset_index(drop=True)

# fig_1 = px.bar(y=mask_budget['расходы на соц. политику'],
#                title='Расходы на соц. политику в рублях',
#                x=mask_budget['Регионы'],
#                color=mask_budget['Федеральный округ'],
#                color_discrete_map=dct_colors,
#                text=mask_budget['метка региона'],
#                height=550,
#                width=1200,
#                range_y=[0,600000000])

# fig_1.update_xaxes(categoryorder='array',categoryarray=mask_budget['Регионы'])
# fig_1.update_traces(textposition='outside',textfont_color='black')
# fig_1.update_layout(legend_orientation='h',
#                     legend=dict(x=0, y=1.2,yanchor='top',title=''),
#                     autosize=True,margin=dict(t=100),
#                     xaxis_title="Регионы",
#                     yaxis_title="Руб.")                   

# fig_1.show()

# fig_2 = px.bar(y=mask_budget['расходы на соц.пол. от бюджета %'],
#                title='расходы на соц.пол. от бюджета %',
#                x=mask_budget['Регионы'],
#                color=mask_budget['Федеральный округ'],
#                color_discrete_map=dct_colors,
#                text=mask_budget['метка региона'],
#                height=550,
#                width=1200,
#                range_y=[0,40])

# fig_2.update_xaxes(categoryorder='array',categoryarray=mask_budget_perc['Регионы'])
# fig_2.update_traces(textposition='outside',textfont_color='black')
# fig_2.update_layout(legend_orientation='h',
#                     legend=dict(x=0, y=1.2,yanchor='top',title=''),
#                     autosize=True,margin=dict(t=100),
#                     xaxis_title="Регионы",
#                     yaxis_title="%") 

# fig_2.show()

В процентном соотношении конечно заметно, что Северо-Кавказкий федеральный округ тратит больше на социальную политику из бюджета, а Москва чуть ли не последняя по процентам, но вот в чистом виде, конечно бюджет Москвы не сравнится ни с чьим другим.

## среднедушевые и реальные денежные доходы населения, номинальная и реальная начисленная зарплата, по регионам*.

In [ ]:
# среднедушевые денежные доходы населения
per_capita_cash_income_2019_2020 = pd.read_excel(
                io='./social_russia_data/cash_real_income_wages_2015_2020.xlsx',
                usecols=[0,6],
                names=['region','Среднедушевые доходы 2020'],
                sheet_name=0)

per_capita_cash_income_2019_2020['region'] = per_capita_cash_income_2019_2020['region'].apply(lambda x: x.strip().replace('\n',' '))
per_capita_cash_income_2019_2020['region'] = per_capita_cash_income_2019_2020['region'].apply(lambda x: dct_city[x] if x in dct_city else x)


# файл который нам пригодится в конце 

per_capita_cash_income_2019_2020 = per_capita_cash_income_2019_2020.drop([23,24,69,70]).reset_index(drop=True)
per_capita_cash_income_2019_2020

In [ ]:
set(data['Регионы']).difference(set(per_capita_cash_income_2019_2020['region']))

А тут нам необходимо понимать, что Архангельская область бедет во внимание и Ненецкий автономный округ. Поэтому их удаляем.

In [ ]:
data = data.merge(right=per_capita_cash_income_2019_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

In [ ]:
# реальные денежные доходы населения
Real_income_2020 = pd.read_excel('./social_russia_data/cash_real_income_wages_2015_2020.xlsx',sheet_name=1,usecols=[0,6])
Real_income_2020 = Real_income_2020.rename(columns={2020:'реальные доходы'}) 
Real_income_2020['region'] = Real_income_2020['region'].apply(lambda x: x.strip().replace('\n',' '))
Real_income_2020['region'] = Real_income_2020['region'].apply(lambda x: dct_city[x] if x in dct_city else x)
Real_income_2020 = Real_income_2020.drop([23,24,69,70]).reset_index(drop=True)
Real_income_2020 

In [ ]:
set(data['Регионы']).difference(set(Real_income_2020['region']))

In [ ]:
data = data.merge(right=Real_income_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

In [ ]:
#data['реальные доходы 2020'] = np.round(data['Среднедушевые доходы 2019']*data['реальные доходы']/100)
# data = data.drop(['Среднедушевые доходы 2019','реальные доходы'],axis=1)
#data

In [ ]:
# mask_income_1 = data.sort_values(by='Среднедушевые доходы 2020',ascending=False)

# fig = px.bar(x=mask_income_1['Регионы'],
#              y=mask_income_1['реальные доходы 2020'],
#              barmode='overlay',
#              text=mask_income_1['метка региона'],
#              title='Среднедушевые доходы',
#              width=1200,
#              height=560)

# for group in mask_income_1['Федеральный округ'].unique():
#     mask = mask_income_1[mask_income_1['Федеральный округ']==group]
#     fig.add_trace(
#                   trace=go.Bar(x=mask['Регионы'],
#                   y=mask['Среднедушевые доходы 2020'],
#                   showlegend=True,
#                   name=group,
#                   legend='legend1',
#                   marker_color = dct_colors[group]))
# fig.update_traces(textposition='outside',textfont_color='#3F51B5')    
# fig.update_layout(xaxis_title="Регионы",
#                   yaxis_title="Руб.")  
# fig.show()

График отчетливо показывает, где самые высокие доходы на душу населения. 

Среднедушевой доход — это показатель уровня жизни семьи, рассчитываемый как средняя сумма доходов всех членов семьи за определенный период (обычно 3 или 12 месяцев), деленная на количество этих членов семьи.

Реальные доходы — это показатель покупательной способности денежных средств, который отражает количество товаров и услуг, которые можно приобрести на полученный доход с учетом инфляции и налогов. 

В Севастополе мы видим, что на те доходы, которые население получает в 2020 году, можно купить такую же корзину товаров, которую он мог себе позволить в прошлом году. С учетом инфляции в остальных регионах на доходы 2020 года приобретешь товара меньше, чем в 2019. Товары с каждым годом дорожают, а рубль падает. Про округа, отчетливо видно, что на урале выделяется только один регион с высокой зп. Ямало-Ненецкий автономный округ в силу того, что там все завязано на добыче нефти и газа. Но тоже надо понимать, такие регионы находятся в тяжелых местностях для доставки  продуктов и проживания, поэтому за все это доплачивают. А так по доходам Москва на 4м месте и Дальневосточный федеральный округ занимает лидирующие места. Тут все связано с расположением и с добычей природных ресурсов, а так же вредностью для здоровья.

In [ ]:
# номинальная начисленная зарплата, по регионам.
Nominal_accrued_salary_2020 = pd.read_excel('./social_russia_data/cash_real_income_wages_2015_2020.xlsx',sheet_name=2,usecols=[0,6])

Nominal_accrued_salary_2020.columns = ['region','номинальная зарплата 2020']

Nominal_accrued_salary_2020['region'] = Nominal_accrued_salary_2020['region'].apply(lambda x: x.strip().replace('\n',' '))
Nominal_accrued_salary_2020['region'] = Nominal_accrued_salary_2020['region'].apply(lambda x: dct_city[x] if x in dct_city else x)
Nominal_accrued_salary_2020 = Nominal_accrued_salary_2020.drop([23,24,69,70]).reset_index(drop=True)

Nominal_accrued_salary_2020


In [ ]:
data = data.merge(right=Nominal_accrued_salary_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

In [ ]:
# Реальная начисленная зарплата по регионам
Real_accrued_salary_2020 = pd.read_excel('./social_russia_data/cash_real_income_wages_2015_2020.xlsx',sheet_name=3,usecols=[0,6])

Real_accrued_salary_2020 = Real_accrued_salary_2020.rename(columns={2020:'начисленная зп. реал. 2020'})
Real_accrued_salary_2020['region'] = Real_accrued_salary_2020['region'].apply(lambda x: x.strip().replace('\n',' '))
Real_accrued_salary_2020['region'] = Real_accrued_salary_2020['region'].apply(lambda x: dct_city[x] if x in dct_city else x)
Real_accrued_salary_2020 = Real_accrued_salary_2020.drop([23,24,69,70]).reset_index(drop=True)

Real_accrued_salary_2020

In [ ]:
set(data['Регионы']).difference(set(Real_accrued_salary_2020['region']))

In [ ]:
data = data.merge(right=Real_accrued_salary_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

In [ ]:
#data['реальная начисленная зп 2020'] = np.round(data['номинальная зарплата 2019'] * data['начисленная зп. реал. 2020']/100,2)
#data = data.drop(['номинальная зарплата 2019','начисленная зп. реал. 2020'],axis=1)
#data

In [ ]:
# mask_income_1 = data.sort_values(by='номинальная зарплата 2020',ascending=False)

# fig = px.bar(x=mask_income_1['Регионы'],
#              y=mask_income_1['реальная начисленная зп 2020'],
#              barmode='overlay',
#              text=mask_income_1['метка региона'],
#              title='номинальная зарплата 2020',
#              width=1200,
#              height=560)

# for group in mask_income_1['Федеральный округ'].unique():
#     mask = mask_income_1[mask_income_1['Федеральный округ']==group]
#     fig.add_trace(
#                   trace=go.Bar(x=mask['Регионы'],
#                   y=mask['номинальная зарплата 2020'],
#                   showlegend=True,
#                   name=group,
#                   legend='legend1',
#                   marker_color = dct_colors[group]))
# fig.update_traces(textposition='outside',textfont_color='#3F51B5')    
# fig.update_layout(xaxis_title="Регионы",
#                   yaxis_title="Руб.")  
# fig.show()

In [ ]:
# mask_income = data.sort_values(by='номинальная зарплата 2020',ascending=False)
# fig = go.Figure(layout=dict(barmode='overlay',
#                             height=530,
#                             width=1200,
#                             xaxis_title="Регионы",
#                             yaxis_title="Руб.",
#                             title='Сравнительный анализ доходов',
#                             legend1=dict(x=1.1,y=1.),
#                             legend2=dict(x=1.1,y=0.75,title='Среднедушевой доход')))



# fig.add_bar(x=mask_income['Регионы'],
#             y=mask_income['номинальная зарплата 2020'],
#             showlegend=True,
#             name='Номинальная зарплата',
#             legend='legend1',
#             text=mask_income['метка региона'],
#             marker_color='black')


# for group in mask_income['Федеральный округ'].unique():
#     mask = mask_income[mask_income['Федеральный округ']==group]
#     fig.add_trace(trace=go.Bar(x=mask['Регионы'],
#                   y=mask['Среднедушевые доходы 2020'],
#                   showlegend=True,
#                   name=group,
#                   legend='legend2',
#                   marker_color = dct_colors[group])  )




# fig.update_xaxes(categoryorder='array', categoryarray=mask_income['Регионы'])
# fig.update_traces(textposition='outside',textfont_color='#3F51B5')
# fig.show()

In [ ]:
dct_city.update({
'Bладимирская область':'Владимирская область',
'Bолгоградская область':'Волгоградская область',
'Bологодская область':'Вологодская область',
'Bоронежская область':'Воронежская область',
'г.Москва':'Москва',
'Еврейская авт. область':'Еврейская автономная область',
'Ненецкий авт. округ':'Ненецкий АО.(Арх.обл.)',
'Республика Саха(Якутия)':'Республика Саха (Якутия)',
'Ханты-Мансийский авт. округ':'ХМАО(Тюменская обл.)',
'Чукотский авт. округ':'Чукотский автономный округ',
'Ямало-Ненецкий авт. округ':'ЯНАО(Тюменская обл.)'})

In [ ]:
# характеристика жилищных условий домохозяйств. Оценка домохозяйствами состояния занимаемого ими жилого помещения, обследование 2020 года*.

data_housing_2020_cond = pd.read_excel('./social_russia_data/housing_2020.xlsx',sheet_name='housing_cond',usecols=[0]+list(range(6,9)))

data_housing_2020_cond.rename(columns={'Регион':'region'},inplace=True)
data_housing_2020_cond['region'] = data_housing_2020_cond['region'].apply(lambda x: x.strip().replace('\n',' '))
data_housing_2020_cond['region'] = data_housing_2020_cond['region'].apply(lambda x: dct_city[x] if x in dct_city else x)
# Этот файл нам нужен будет для аналитики 2020 г.
# data_housing_2020_cond = data_housing_2020_cond.iloc[:,[0,6,7,8]]
data_housing_2020_cond

In [ ]:

data = data.merge(right=data_housing_2020_cond,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

In [ ]:
data_housing_2020_intent = pd.read_excel('./social_russia_data/housing_2020.xlsx',sheet_name='housing_intent')
data_housing_2020_intent.rename(columns={'Регион':'region'},inplace=True)
data_housing_2020_intent.drop([94,95],inplace=True)
data_housing_2020_intent['region'] = data_housing_2020_intent['region'].apply(lambda x: x.replace('\n',' '))
data_housing_2020_intent.replace(r'(\.{2,}\*)|(\.{2,})|(\…\*)|(\…)',0,regex=True,inplace=True)
data_housing_2020_intent['region'] = data_housing_2020_intent['region'].apply(lambda x: dct_city[x] if x in dct_city else x)
data_housing_2020_intent

In [ ]:
set(data['Регионы']).difference(set(data_housing_2020_intent['region']))

In [ ]:
# data = data.merge(right=data_housing_2020_intent,
#                   how='left',
#                   left_on='Регионы',
#                   right_on='region').drop(['region'],axis=1)
# data

$$\frac{\text{ВВП}}{\text{численность населения}}$$

$$\text{ВВП на душу населения} = \frac{180,000,000,000,000}{146,000,000} \approx 1,232,876 , (\text{рублей}) $$

In [ ]:
# валовой региональный продукт на душу населения, в рублях.

data_gross_regional_product_1996_2020 = pd.read_excel('./social_russia_data/gross_regional_product_1996_2020.xls')
data_gross_regional_product_2020 = data_gross_regional_product_1996_2020.iloc[2:,[0,-1]]
data_gross_regional_product_2020.columns = ['region','Валовый продукт']
data_gross_regional_product_2020['region'] = data_gross_regional_product_2020['region'].apply(lambda x: x.strip())
# Этот файл нам нужен будет для аналитики 2020 г.
data_gross_regional_product_2020 = data_gross_regional_product_2020.drop([25,72]).reset_index(drop=True)
data_gross_regional_product_2020 


In [ ]:
data_gross_regional_product_2020['region'] = data_gross_regional_product_2020['region'].apply(lambda x: dct_city[x] if x in dct_city else x)

In [ ]:
set(data['Регионы']).difference(set(data_gross_regional_product_2020['region']))

In [ ]:
data = data.merge(right=data_gross_regional_product_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

In [ ]:
# объём отгруженных товаров собственного производства или работ/услуг, выполненных собственными силами, 
# по видам деятельности за 2005–2016 гг. 2017–2020 гг. (в тысячах рублей, значение показателя за год, полный круг)

data_regional_production_2005_2016 = pd.read_csv('./social_russia_data/regional_production_2005_2016.csv')
data_regional_production_2017_2020 = pd.read_csv('./social_russia_data/regional_production_2017_2020.csv')
data_regional_production_2017_2020.head()
data_regional_production_2020 = data_regional_production_2017_2020.groupby('region')['2020'].sum().reset_index()
data_regional_production_2020 = data_regional_production_2020.rename(columns={'2020':'объем собствен. производства'})
data_regional_production_2020['region'] = data_regional_production_2020['region'].apply(lambda x:x.strip())
# Этот файл нам нужен будет для аналитики 2020 г.
data_regional_production_2020

In [ ]:
data_regional_production_2020 = data_regional_production_2020.drop(index=[7,78]).reset_index(drop=True)
data_regional_production_2020

In [ ]:
data_regional_production_2020['region'] = data_regional_production_2020['region'].apply(lambda x: dct_city[x] if x in dct_city else x)

In [ ]:
set(data['Регионы']).difference(set(data_regional_production_2020['region']))

In [ ]:
data = data.merge(right=data_regional_production_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

In [ ]:
# оборот розничной торговли на душу населения, в рублях.

data_retail_turnover_per_capita_2000_2021 = pd.read_excel('./social_russia_data/retail_turnover_per_capita_2000_2021.xls')
data_retail_turnover_per_capita_2020 = data_retail_turnover_per_capita_2000_2021.iloc[3:,[0,22]]
data_retail_turnover_per_capita_2020.columns = ['region','оборот розничной торговли на душу населения']
data_retail_turnover_per_capita_2020['region'] = data_retail_turnover_per_capita_2020['region'].apply(lambda x:x.strip())
# Этот файл нам нужен будет для аналитики 2020 г.
data_retail_turnover_per_capita_2020

In [ ]:
data_retail_turnover_per_capita_2020 = data_retail_turnover_per_capita_2020.drop(index=[26,28,74,77])

In [ ]:
data_retail_turnover_per_capita_2020['region'] = data_retail_turnover_per_capita_2020['region'].apply(lambda x: dct_city[x] if x in dct_city else x)

In [ ]:
set(data['Регионы']).difference(set(data_retail_turnover_per_capita_2020['region']))

In [ ]:
data = data.merge(right=data_retail_turnover_per_capita_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

In [ ]:
# сведения о заболеваемости алкоголизмом на 100 тыс. населения (2005–2018)

data_alco_2018 = pd.read_excel(io='./social_russia_data/drug_alco.xlsx',sheet_name='alco1718',usecols=[0,2],names=['region','алкоголизм 2018'])
data_alco_2018


# Этот файл нам нужен будет для аналитики 2020 г.

In [ ]:
dct_city.update({'город Санкт - Петербург':'Санкт-Петербург','Тюменская область без автономного округа':'Тюменская обл.(кроме ХМАО,ЯНАО)',
                 'Ханты-Мансийский АО':'ХМАО(Тюменская обл.)','Ямало-Hенецкий АО':'ЯНАО(Тюменская обл.)'})

In [ ]:
data_alco_2018['region'] = data_alco_2018['region'].apply(lambda x: dct_city[x] if x in dct_city else x)

In [ ]:
set(data['Регионы']).difference(set(data_alco_2018['region']))

In [ ]:
# data = data.merge(right=data_alco_2018,
#                   how='left',
#                   left_on='Регионы',
#                   right_on='region').drop(['region'],axis=1)
# data

In [ ]:
# сведения о заболеваемости наркоманией на 100 тыс. населения 2018

data_drug_2018 = pd.read_excel(io='./social_russia_data/drug_alco.xlsx',sheet_name='drug1718',usecols=[0,2],names=['region',' наркомания 2018'])
data_drug_2018


# Этот файл нам нужен будет для аналитики 2020 г.

In [ ]:
data_drug_2018['region'] = data_drug_2018['region'].apply(lambda x: dct_city[x] if x in dct_city else x)

In [ ]:
# data = data.merge(right=data_drug_2018,
#                   how='left',
#                   left_on='Регионы',
#                   right_on='region').drop(['region'],axis=1)
# data

In [ ]:
data_crimes_2020 = pd.read_excel(io='./social_russia_data/crimes2020.xls',header=11,usecols=[1,2,3,4,5,6,7,8,9,10,11])
data_crimes_2020.columns = ['region','несовершеннолетними или при их соучастии','ранее совершавшими преступления',
                            'в том числе ранее судимыми','группой лиц','группой лиц по предварительному сговору',
                            'организованной группой','преступным сообществом (преступной организацией)',
                            'в состоянии алкогольного опьянения','наркотического опьянения','токсического опьянения']
data_crimes_2020

In [ ]:
dct_city.update({'Кемеровская область – Кузбасс':'Кемеровская область','Новгородская  область':'Новгородская область',
                 'Республика Северная Осетия – Алания':'Республика Северная Осетия-Алания',
                 'Ханты-Мансийский автономный округ –  Югра':'ХМАО(Тюменская обл.)'})

In [ ]:
data_crimes_2020['region'] = data_crimes_2020['region'].apply(lambda x: dct_city[x] if x in dct_city else x)

In [ ]:
set(data['Регионы']).difference(set(data_crimes_2020['region']))

In [ ]:
data = data.merge(right=data_crimes_2020,
                  how='left',
                  left_on='Регионы',
                  right_on='region').drop(['region'],axis=1)
data

In [ ]:
df = data.copy()
df

In [ ]:
(df==0).sum().reset_index()

In [ ]:
dfr = df.drop(labels=df.columns[[0,1,2,3,4,6,7,10,11,12,13,14,15,21,22,23,24,25,49]],axis=1)
dfr

Остановился здесь!!!

Так же нужно понимать, что есть причины и следствия, т.е. причина может быть в площади квартиры, а следствие может быть желание поменять ее. Поэтому стоит удалить из датасета признаки, которые являются следствием.

In [ ]:
corr_matrix = dfr.iloc[:,1:].corr()

threshold = 0.8
high_corr_features = []
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i,j]) > threshold:
            high_corr_features.append((corr_matrix.columns[i], corr_matrix.columns[j]))


dct = dict()

for value1,value2 in high_corr_features:
    if value1 not in dct:
        dct[value1] = 1
    else:
         dct[value1] += 1

    if value2 not in dct:
        dct[value2] = 1
    else:
         dct[value2] += 1

sorted(dct.items(),key=lambda item: item[1],reverse = True)

In [ ]:
data_drop = dfr.drop(['ранее совершавшими преступления',
                      '% 41_50_инвалид',
                      'в том числе ранее судимыми',
                      '% 18_30_инвалид',
                      'номинальная зарплата 2020',
                      'объем собствен. производства',
                      'доля 16-60 за чертой бедности',
                      'Размер жилой площади в расчете на члена домохозяйства',
                      'Валовый продукт',
                      'в состоянии алкогольного опьянения'],axis=1)



corr_matrix = data_drop.corr()

threshold = 0.8
high_corr_features = []
for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i,j]) > threshold:
            high_corr_features.append((corr_matrix.columns[i], corr_matrix.columns[j]))


dct = dict()

for value1,value2 in high_corr_features:
    if value1 not in dct:
        dct[value1] = 1
    else:
         dct[value1] += 1

    if value2 not in dct:
        dct[value2] = 1
    else:
         dct[value2] += 1

sorted(dct.items(),key=lambda item: item[1],reverse = True)

In [ ]:
data_drop

И вроде как хочется отметить, что кластеров всего 4, т.к. и ранее на логте и на оценке кросс-валидации эта оценка одиннаковая

## <center> Кодирование признаков.

Стандартизация данных

In [ ]:
scaler = RobustScaler()
scaler.fit(data_drop)
cols = data_drop.columns
data_scaler = scaler.transform(data_drop)
data_scaler = pd.DataFrame(data_scaler,columns=cols)
data_scaler

In [ ]:
from sklearn.decomposition import PCA
# создаём объект класса PCA
# n_components — задаём количество компонентов для проведения трансформации
pca = PCA(n_components=3, random_state=42)
# обучаем модель на данных X
pca.fit(data_scaler)
# применяем уменьшение размерности к матрице X
pca_data = pca.transform(data_scaler) 
pca_data = pd.DataFrame(pca_data)
pca_data

px.scatter_3d(x=pca_data[0],y=pca_data[1],z=pca_data[2],width=1200,height=400)

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Вычисляем сумму квадратичных отклонений для разных чисел кластеров
inertia = []
for k in range(1, 13):
    km = KMeans(n_clusters=k, random_state=42)
    km.fit(data_scaler)
    inertia.append(km.inertia_)

plt.plot(range(1, 13), inertia, marker='o')
plt.title('Метод локтя')
plt.xlabel('Количество кластеров')
plt.ylabel('Сумма квадратичных отклонений')
plt.show()

In [ ]:
from yellowbrick.cluster import KElbowVisualizer  # Используется только для K-MEANS
Elbow_M = KElbowVisualizer(KMeans(random_state=42), k=10) 
Elbow_M.fit(data_scaler)
Elbow_M.show()

In [ ]:
from sklearn.metrics import davies_bouldin_score
sil_scores = []
for k in range(2, 11):
    km = KMeans(n_clusters=k, random_state=42)
    labels = km.fit_predict(data_scaler)
    score = davies_bouldin_score(data_scaler, labels)
    sil_scores.append(score)

plt.plot(range(2, 11), sil_scores, marker='o')
plt.title('Davies-Bouldin Index')
plt.xlabel('Количество кластеров')
plt.ylabel('индекс')
plt.show()

In [ ]:
k_means =  KMeans(n_clusters=4,random_state=42)
k_means.fit(data_scaler)

k_means.labels_

In [ ]:
px.scatter_3d(x=pca_data[0],y=pca_data[1],z=pca_data[2],width=1200,height=400,color=k_means.labels_)

In [ ]:


px.scatter(x=pca_data[0],y=pca_data[1],width=1200,height=400,color=k_means.labels_)

In [ ]:
data['clusters'] = k_means.labels_
data[data['clusters']==0]

## <center> Построение модели

In [ ]:
from sklearn.cluster import AgglomerativeClustering
agglomerative_clustering = AgglomerativeClustering(n_clusters=4)
agglomerative_clustering.fit(data_scaler)
agglomerative_clustering.labels_



In [ ]:

data['clusters'] = agglomerative_clustering.labels_
data[data['clusters']==2]

In [ ]:
from sklearn.cluster import DBSCAN

# Запускаем кластеризацию на наших данных
model_2 = DBSCAN(eps=0.5, min_samples=4)

model_2.fit(data_scaler)

px.scatter_3d(x=pca_data[0],y=pca_data[1],z=pca_data[2],width=1200,height=400,color=model_2.labels_)
